# Automated Splitting/Tagging

### Data preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%bash
cd '/content/drive/My Drive/Coterm/CS224V'

In [ ]:
import pandas as pd

In [ ]:
transcript_file_path = '/content/drive/My Drive/Coterm/CS224V/cbm_biweekly_transcripts/'
raw_df = pd.read_csv(transcript_file_path + '08.13mtg-Table 1.csv')
raw_df1 = pd.read_csv(transcript_file_path + '08.27mtg-Table 1.csv')
raw_df2 = pd.read_csv(transcript_file_path + '07.30mtg-Table 1.csv')
raw_df3 = pd.read_csv(transcript_file_path + '07.16mtg-Table 1.csv')
raw_df_concat = pd.concat([raw_df, raw_df1, raw_df2, raw_df3], axis=0, ignore_index=True)
# df.fillna('NA', inplace=True)
# raw_df_concat = raw_df_concat.dropna(subset=['speaker'])
raw_df_concat = raw_df_concat.drop(raw_df_concat.columns[-1], axis=1)
print(raw_df_concat.columns.tolist())



['turn', 'start time', 'speaker', 'speaker turn']


In [ ]:
raw_df_concat.tail(10)

,turn,start time,speaker,speaker turn
1099,215,1:19:35,M1,"Not, not so far."
1100,216,1:19:37,L0,"Okay, thanks. And anything else from A1?"
1101,217,1:19:45,A1,Nope.
1102,218,1:19:46,L0,"Nope, we’re good. All right, how about you A0?"
1103,219,1:19:50:00.000,A0,Good.
1104,220,1:19:51,L0,All right.
1105,221,1:19:52,A0,I know my action items.
1106,222,1:19:54,L0,So I write up the meeting minutes and go from ...
1107,223,1:20:02,A0,Cool
1108,224,1:20:03:00.000,M1,Thank you.


In [ ]:
### Generate/preprocess the gold dataset
gold_transcript_file_path = '/content/drive/My Drive/Coterm/CS224V/Bi-Weekly-Labeled/'
# gold_df = pd.read_csv(gold_transcript_file_path)

df = pd.read_csv(gold_transcript_file_path + '8.13mtg.csv')
df1 = pd.read_csv(gold_transcript_file_path + '8.27mtg.csv')
df2 = pd.read_csv(gold_transcript_file_path + '7.30mtg.csv')
df3 = pd.read_csv(gold_transcript_file_path + '7.16mtg.csv')

# gold_df = pd.concat([df, df1, df2, df3], axis=0, ignore_index=True)

df = df.iloc[:, :5]
df.fillna("NA", inplace=True)
# df = df.dropna(subset=['speaker', 'speech act'])

df1 = df1.iloc[:, :5]
df1.fillna("NA", inplace=True)
# df1 = df1.dropna(subset=['speaker', 'speech act'])

df2 = df2.iloc[:, :5]
df2.fillna("NA", inplace=True)
# df2 = df2.dropna(subset=['speaker', 'speech act'])

df3 = df3.iloc[:, :5]
df3.fillna("NA", inplace=True)
# df3 = df3.dropna(subset=['speaker', 'speech act'])

print(df.columns.tolist())
df.head(25)


['turn', 'start time', 'speaker', 'utterance', 'speech act']


,turn,start time,speaker,utterance,speech act
0,1.0,0:04,L0,No problem. [to me about recording],NA
1,2.0,0:05,NA,[silence],NA
2,3.0,2:08,A2,[overheard phone conversation] don’t come back...,NA
3,3.0,2:35,A2,[gives thumbs up],NA
4,4.0,2:36,L0,"Hey A2, how’s it going?",Directive
5,5.0,2:38,NA,[silence],NA
6,6.0,2:49,L0,"Hi M1 did you see uh M0 today? Is he ah, do yo...",Directive
7,7.0,2:57,M1,"Um, I’m not on site today, but he has sent an ...",Assertive
8,8.0,3:03,L0,Ok All right.,Assertive
9,8.0,3:03,L0,I’ll give him a few more minutes see if M0 can...,Commissive


In [ ]:
def concatenate_utterance_speech_act(group):
  last_speech_act = None
  concat_str = ""
  for index in reversed(range(len(group))):
    row = group.iloc[index]
    prefix_str = row['utterance'] + ' '
    if row['speech act'] != last_speech_act:
      last_speech_act = row['speech act']
      prefix_str += '<' + row['speech act'] + '> '
    concat_str = prefix_str + concat_str

  return concat_str
  # return ' '.join(group['utterance'] + ' <' + group['speech act'] + '>')

# df_test = df[df['turn'] == 19]
# print(df_test)
grouped_df = df.groupby(['turn', 'speaker']).apply(concatenate_utterance_speech_act).reset_index(name='speaker turn gold')
grouped_df1 = df1.groupby(['turn', 'speaker']).apply(concatenate_utterance_speech_act).reset_index(name='speaker turn gold')
grouped_df2 = df2.groupby(['turn', 'speaker']).apply(concatenate_utterance_speech_act).reset_index(name='speaker turn gold')
grouped_df3 = df3.groupby(['turn', 'speaker']).apply(concatenate_utterance_speech_act).reset_index(name='speaker turn gold')

gold_df = pd.concat([grouped_df, grouped_df1, grouped_df2, grouped_df3], axis=0, ignore_index=True)

gold_df.head(20)
# grouped_df.head(20)



,turn,speaker,speaker turn gold
0,1.0,L0,No problem. [to me about recording] <NA>
1,2.0,NA,[silence] <NA>
2,3.0,A2,[overheard phone conversation] don’t come back...
3,4.0,L0,"Hey A2, how’s it going? <Directive>"
4,5.0,NA,[silence] <NA>
5,6.0,L0,"Hi M1 did you see uh M0 today? Is he ah, do yo..."
6,7.0,M1,"Um, I’m not on site today, but he has sent an ..."
7,8.0,L0,Ok All right. <Assertive> I’ll give him a few ...
8,9.0,NA,[silence] <NA>
9,10.0,E0,L0 are you still are we still waiting for some...


In [ ]:
gold_df = gold_df.reset_index(drop=True)
raw_df_concat = raw_df_concat.reset_index(drop=True)
gold_df['speaker turn'] = raw_df_concat['speaker turn']
# drop the na rows
# df = df.dropna(subset=['speaker', 'speech act'])
gold_df = gold_df[~gold_df.applymap(lambda x: '<NA>' in str(x)).any(axis=1)]
gold_df.tail(10)

,turn,speaker,speaker turn gold,speaker turn
1102,215.0,M1,"Not, not so far. <Assertive>","Nope, we’re good. All right, how about you A0?"
1103,216.0,L0,"Okay, thanks. <Expressive> And anything else ...",Good.
1104,217.0,A1,Nope. <Assertive>,All right.
1105,218.0,L0,"Nope, we’re good. All right, how about you A0?...",I know my action items.
1106,219.0,A0,Good. <Assertive>,So I write up the meeting minutes and go from ...
1107,220.0,L0,All right. <Assertive>,Cool
1108,221.0,A0,I know my action items. <Assertive>,Thank you.
1109,222.0,L0,So I write up the meeting minutes and go from ...,NaN
1110,223.0,A0,Cool <Assertive>,NaN
1111,224.0,M1,Thank you. <Expressive>,NaN


In [ ]:
gold_df.to_csv('/content/drive/My Drive/Coterm/CS224V/gold_train_meetings.csv', index=False)

In [ ]:
df = df.reset_index(drop=True)
grouped_gold_df = grouped_gold_df.reset_index(drop=True)
df['speaker turn gold'] = grouped_gold_df['speaker turn gold']
df.head(10)

,start time,turn,speaker,speaker turn,spearker turn gold,speaker turn gold
0,4:54:00,2,A1,"guys, I can see we're still waiting. We’re sti...","guys, I can see we're still waiting. We’re sti...","guys, I can see we're still waiting. We’re sti..."
1,5:02:00,3,L0,sure,sure <Assertive>,sure <Assertive>
2,5:02:00,4,A1,in two minutes.,in two minutes. <Commissive>,in two minutes. <Commissive>
3,5:48:00,6,A1,[sighs or lets out air] ok. So L0 uh there’s o...,[sighs or lets out air] ok. So L0 uh there’s o...,[sighs or lets out air] ok. So L0 uh there’s o...
4,6:09:00,7,L0,Oh,Oh <Assertive>,Oh <Assertive>
5,6:09:00,8,A1,"So I will, I will start testing on the develop...","So I will, I will start testing on the develop...","So I will, I will start testing on the develop..."
6,6:50:00,9,L0,yeah um so this this Software A CMMS integrati...,yeah um so this this Software A CMMS integrati...,yeah um so this this Software A CMMS integrati...
7,7:01:00,10,A1,yeah. So…,yeah. So… <Assertive>,yeah. So… <Assertive>
8,7:04:00,11,L0,So. yeah go ahead,So. yeah go ahead <Directive>,So. yeah go ahead <Directive>
9,7:06:00,12,A1,"So the way it works is, we will, Software A wi...","So the way it works is, we will, Software A wi...","So the way it works is, we will, Software A wi..."


In [ ]:
df = df.drop('spearker turn gold', axis=1)
df.head(5)

,start time,turn,speaker,speaker turn,speaker turn gold
0,4:54:00,2,A1,"guys, I can see we're still waiting. We’re sti...","guys, I can see we're still waiting. We’re sti..."
1,5:02:00,3,L0,sure,sure <Assertive>
2,5:02:00,4,A1,in two minutes.,in two minutes. <Commissive>
3,5:48:00,6,A1,[sighs or lets out air] ok. So L0 uh there’s o...,[sighs or lets out air] ok. So L0 uh there’s o...
4,6:09:00,7,L0,Oh,Oh <Assertive>


In [ ]:
df.to_csv('/content/drive/My Drive/Coterm/CS224V/9.10mtg_preprocessed.csv', index=False)
preprocessed_df = df

In [ ]:
### One time run
cur_df = pd.read_csv('/content/drive/My Drive/Coterm/CS224V/9.10mtg_preprocessed.csv')
cur_df = cur_df.dropna(subset=['speaker turn'])
cur_df.tail(5)
cur_df.to_csv('/content/drive/My Drive/Coterm/CS224V/9.10mtg_preprocessed.csv', index=False)

###**PREPARING DATA IN FOR WINSTON**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
transcript_file_path = '/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/'
raw_df = pd.read_csv(transcript_file_path + '08.13mtg-Table 1.csv')
raw_df1 = pd.read_csv(transcript_file_path + '08.27mtg-Table 1.csv')
raw_df2 = pd.read_csv(transcript_file_path + '07.30mtg-Table 1.csv')
raw_df3 = pd.read_csv(transcript_file_path + '07.16mtg-Table 1.csv')
raw_df_concat = pd.concat([raw_df, raw_df1, raw_df2, raw_df3], axis=0, ignore_index=True)
# df.fillna('NA', inplace=True)
# raw_df_concat = raw_df_concat.dropna(subset=['speaker'])
raw_df_concat = raw_df_concat.drop(raw_df_concat.columns[-1], axis=1)

print(len(raw_df))
print(len(raw_df1))
print(len(raw_df2))
print(len(raw_df3))

325
277
283
224


In [ ]:
### Generate/preprocess the gold dataset
gold_transcript_file_path = '/content/drive/MyDrive/Coterm/CS224V/Bi-Weekly-Labeled/'
# gold_df = pd.read_csv(gold_transcript_file_path)

df = pd.read_csv(gold_transcript_file_path + '8.13mtg.csv')
df1 = pd.read_csv(gold_transcript_file_path + '8.27mtg.csv')
df2 = pd.read_csv(gold_transcript_file_path + '7.30mtg.csv')
df3 = pd.read_csv(gold_transcript_file_path + '7.16mtg.csv')

# gold_df = pd.concat([df, df1, df2, df3], axis=0, ignore_index=True)

df = df.iloc[:, :5]
df.fillna("NA", inplace=True)
# df = df.dropna(subset=['speaker', 'speech act'])

df1 = df1.iloc[:, :5]
df1.fillna("NA", inplace=True)
# df1 = df1.dropna(subset=['speaker', 'speech act'])

df2 = df2.iloc[:, :5]
df2.fillna("NA", inplace=True)
# df2 = df2.dropna(subset=['speaker', 'speech act'])

df3 = df3.iloc[:, :5]
df3.fillna("NA", inplace=True)
# df3 = df3.dropna(subset=['speaker', 'speech act'])

print(len(df))
print(len(df1))
print(len(df2))
print(len(df3))
df.tail(10)

416
363
386
362


,turn,start time,speaker,utterance,speech act
406,320.0,46:46:00,M0,"Thank you,",Expressive
407,321.0,46:46:00,L0,thank you,Expressive
408,322.0,46:47:00,A0,yup,Assertive
409,323.0,46:47:00,E0,Thank you guys,Expressive
410,324.0,46:48:00,A2,Thanks everyone.,Expressive
411,325.0,46:49:00,L0,bye,Expressive
412,NA,NA,NA,NA,NA
413,NA,NA,NA,NA,NA
414,NA,NA,NA,NA,NA
415,NA,NA,NA,NA,NA


In [ ]:
def concatenate_utterance_speech_act(group):
  last_speech_act = None
  concat_str = ""
  for index in reversed(range(len(group))):
    row = group.iloc[index]
    prefix_str = row['utterance'] + ' '
    if row['speech act'] != last_speech_act:
      last_speech_act = row['speech act']
      prefix_str += '<' + row['speech act'] + '> '
    concat_str = prefix_str + concat_str

  return concat_str

grouped_df = df.groupby(['turn', 'speaker']).apply(concatenate_utterance_speech_act).reset_index(name='speaker turn gold')
grouped_df = grouped_df.drop(grouped_df.index[-1])

grouped_df1 = df1.groupby(['turn', 'speaker']).apply(concatenate_utterance_speech_act).reset_index(name='speaker turn gold')
grouped_df1 = grouped_df1.drop(grouped_df1.index[-1])

grouped_df2 = df2.groupby(['turn', 'speaker']).apply(concatenate_utterance_speech_act).reset_index(name='speaker turn gold')
grouped_df2 = grouped_df2.drop(grouped_df2.index[-1])

grouped_df3 = df3.groupby(['turn', 'speaker']).apply(concatenate_utterance_speech_act).reset_index(name='speaker turn gold')
grouped_df3 = grouped_df3.drop(grouped_df3.index[-1])

gold_df = pd.concat([grouped_df, grouped_df1, grouped_df2, grouped_df3], axis=0, ignore_index=True)

print(len(grouped_df))
print(len(grouped_df1))
print(len(grouped_df2))
print(len(grouped_df3))

325
277
283
224


In [ ]:
gold_df = gold_df.reset_index(drop=True)
raw_df_concat = raw_df_concat.reset_index(drop=True)
gold_df['speaker turn'] = raw_df_concat['speaker turn']
# drop the na rows
# df = df.dropna(subset=['speaker', 'speech act'])
gold_df.tail(10)

,turn,speaker,speaker turn gold,speaker turn
1099,215.0,M1,"Not, not so far. <Assertive>","Not, not so far."
1100,216.0,L0,"Okay, thanks. <Expressive> And anything else ...","Okay, thanks. And anything else from A1?"
1101,217.0,A1,Nope. <Assertive>,Nope.
1102,218.0,L0,"Nope, we’re good. All right, how about you A0?...","Nope, we’re good. All right, how about you A0?"
1103,219.0,A0,Good. <Assertive>,Good.
1104,220.0,L0,All right. <Assertive>,All right.
1105,221.0,A0,I know my action items. <Assertive>,I know my action items.
1106,222.0,L0,So I write up the meeting minutes and go from ...,So I write up the meeting minutes and go from ...
1107,223.0,A0,Cool <Assertive>,Cool
1108,224.0,M1,Thank you. <Expressive>,Thank you.


In [ ]:
gold_df.head(10)

,turn,speaker,speaker turn gold,speaker turn
0,1.0,L0,No problem. [to me about recording] <NA>,No problem. [to me about recording]
1,2.0,NA,[silence] <NA>,[silence]
2,3.0,A2,[overheard phone conversation] don’t come back...,[overheard phone conversation] don’t come back...
3,4.0,L0,"Hey A2, how’s it going? <Directive>","Hey A2, how’s it going?"
4,5.0,NA,[silence] <NA>,[silence]
5,6.0,L0,"Hi M1 did you see uh M0 today? Is he ah, do yo...","Hi M1 did you see uh M0 today? Is he ah, do yo..."
6,7.0,M1,"Um, I’m not on site today, but he has sent an ...","Um, I’m not on site today, but he has sent an ..."
7,8.0,L0,Ok All right. <Assertive> I’ll give him a few ...,Ok All right. I’ll give him a few more minutes...
8,9.0,NA,[silence] <NA>,[silence]
9,10.0,E0,L0 are you still are we still waiting for some...,L0 are you still are we still waiting for some...


In [ ]:
gold_df = gold_df[~gold_df.applymap(lambda x: '<NA>' in str(x)).any(axis=1)]
gold_df.tail(10)

,turn,speaker,speaker turn gold,speaker turn
1099,215.0,M1,"Not, not so far. <Assertive>","Not, not so far."
1100,216.0,L0,"Okay, thanks. <Expressive> And anything else ...","Okay, thanks. And anything else from A1?"
1101,217.0,A1,Nope. <Assertive>,Nope.
1102,218.0,L0,"Nope, we’re good. All right, how about you A0?...","Nope, we’re good. All right, how about you A0?"
1103,219.0,A0,Good. <Assertive>,Good.
1104,220.0,L0,All right. <Assertive>,All right.
1105,221.0,A0,I know my action items. <Assertive>,I know my action items.
1106,222.0,L0,So I write up the meeting minutes and go from ...,So I write up the meeting minutes and go from ...
1107,223.0,A0,Cool <Assertive>,Cool
1108,224.0,M1,Thank you. <Expressive>,Thank you.


In [ ]:
gold_df.to_csv('/content/drive/MyDrive/Coterm/CS224V/gold_train_meetings.csv', index=False)

In [ ]:
finetuning_df = pd.read_csv('/content/drive/MyDrive/Coterm/CS224V/gold_train_meetings.csv')
finetuning_df.head(10)

,turn,speaker,speaker turn gold,speaker turn
0,4.0,L0,"Hey A2, how’s it going? <Directive>","Hey A2, how’s it going?"
1,6.0,L0,"Hi M1 did you see uh M0 today? Is he ah, do yo...","Hi M1 did you see uh M0 today? Is he ah, do yo..."
2,7.0,M1,"Um, I’m not on site today, but he has sent an ...","Um, I’m not on site today, but he has sent an ..."
3,8.0,L0,Ok All right. <Assertive> I’ll give him a few ...,Ok All right. I’ll give him a few more minutes...
4,10.0,E0,L0 are you still are we still waiting for some...,L0 are you still are we still waiting for some...
5,11.0,L0,yeah I'm waiting for M0 and A1. <Assertive>,yeah I'm waiting for M0 and A1.
6,12.0,E0,And then we're recording this. <Directive>,And then we're recording this.
7,13.0,L0,yeah we're. we're recording these <Assertive>,yeah we're. we're recording these
8,14.0,E0,Let’s look our best <Directive>,Let’s look our best
9,15.0,L0,But if they can’t join i'm thinking about canc...,But if they can’t join i'm thinking about canc...


In [ ]:
# {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
# {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
# {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}

finetuned_examples = []
system_prompt = {"role": "system",
               "content": "You are a chatbot that identifies speech acts within a sequence."}

for index, row in finetuning_df.iterrows():
    row_example = finetuning_df.sample(n=1)
    combined_sequence = []
    input_sequence = {
        "role": "user",
        "content": "Your options for the speech acts are Commissive, Assertive, Directive, Expressive, or Declaration. Here are the definitions for each speech act:\n\
Commissive: Speaker utters U to commit to a future action or future state.\n\
Directive: Speaker attempts to get the Hearer to do something or bring about a future state by uttering U.\n\
Expressive: Speaker expresses a psychological state about specific affairs.\n\
Declaration: Upon speaker uttering U the propositional content of U becomes reality.\n\
Assertive: Speaker utters U to describe an existing state of affairs.\n\
Identify and label the shifts in sentiment as they occur by inserting the speech act in <> in between the shifts in speech act. Only write the original input text with the speech act tokens inserted.\n\
Example Input: " + row_example['speaker turn'].iloc[0] + '\nExample Output: ' + row_example['speaker turn gold'].iloc[0] + "\nInput: " + row['speaker turn'] + "\nOutput: "

    }
    # if input_sequence["content"] is None or input_sequence["content"] == '':
    #   print("missing speaker turn: ", row['speaker turn'])
    #   print("missing speaker turn at row: ", index)

    output_sequence = {
        "role": "assistant",
        "content": row['speaker turn gold']
    }
    # if output_sequence["content"] is None or output_sequence["content"] == '':
    #   print("missing speaker turn gold: ", row['speaker turn'])
    #   print("missing speaker turn gold at row: ", index)

    combined_sequence.append(system_prompt)
    combined_sequence.append(input_sequence)
    combined_sequence.append(output_sequence)
    message = {"messages": combined_sequence}
    finetuned_examples.append(message)


In [ ]:
print(finetuned_examples[-1])

{'messages': [{'role': 'system', 'content': 'You are a chatbot that identifies speech acts within a sequence.'}, {'role': 'user', 'content': 'Your options for the speech acts are Commissive, Assertive, Directive, Expressive, or Declaration. Here are the definitions for each speech act:\nCommissive: Speaker utters U to commit to a future action or future state.\nDirective: Speaker attempts to get the Hearer to do something or bring about a future state by uttering U.\nExpressive: Speaker expresses a psychological state about specific affairs.\nDeclaration: Upon speaker uttering U the propositional content of U becomes reality.\nAssertive: Speaker utters U to describe an existing state of affairs.\nIdentify and label the shifts in sentiment as they occur by inserting the speech act in <> in between the shifts in speech act. Only write the original input text with the speech act tokens inserted.\nExample Input: yeah. um. Are you interested in sitting in that meeting um, M0?\nExample Outpu

In [ ]:
import json

In [ ]:
with open('/content/drive/MyDrive/Coterm/CS224V/finetuning_data.jsonl', 'w') as file:
  for item in finetuned_examples:
    json_string = json.dumps(item)
    file.write(json_string + '\n')

### THE FOLLOWING IS USED TO CHECK VALID FILE FOR OPENAI FINETUNING

In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
### CHECK WHETHER VALID FILE FOR OPENAI FINETUNING
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [ ]:
data_path = "/content/drive/MyDrive/Coterm/CS224V/finetuning_data.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 1079
First example:
{'role': 'system', 'content': 'You are a chatbot that identifies speech acts within a sequence.'}
{'role': 'user', 'content': 'Your options for the speech acts are Commissive, Assertive, Directive, Expressive, or Declaration. Here are the definitions for each speech act:\nCommissive: Speaker utters U to commit to a future action or future state.\nDirective: Speaker attempts to get the Hearer to do something or bring about a future state by uttering U.\nExpressive: Speaker expresses a psychological state about specific affairs.\nDeclaration: Upon speaker uttering U the propositional content of U becomes reality.\nAssertive: Speaker utters U to describe an existing state of affairs.\nIdentify and label the shifts in sentiment as they occur by inserting the speech act in <> in between the shifts in speech act. Only write the original input text with the speech act tokens inserted.\nExample Input: Did we verified the PM cost?\nExample Output: Did we verifi

In [ ]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
            print("example with missing content: ", ex)

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [ ]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [ ]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 207, 1625
mean / median: 313.04911955514365, 267.0
p5 / p95: 217.0, 441.20000000000005

#### Distribution of num_assistant_tokens_per_example:
min / max: 5, 669
mean / median: 31.934198331788693, 16.0
p5 / p95: 7.0, 64.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~337780 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~1013340 tokens


### Finetuning on OpenAI

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
!pip install --upgrade openai

In [ ]:
from openai import OpenAI
import os

In [ ]:
os.environ['OPENAI_API_KEY'] = 'add api key'

In [ ]:
client = OpenAI(
    organization = "add org",
)
client.models.list()

SyncPage[Model](data=[Model(id='text-search-babbage-doc-001', created=1651172509, object='model', owned_by='openai-dev'), Model(id='gpt-3.5-turbo-16k-0613', created=1685474247, object='model', owned_by='openai'), Model(id='curie-search-query', created=1651172509, object='model', owned_by='openai-dev'), Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'), Model(id='text-search-babbage-query-001', created=1651172509, object='model', owned_by='openai-dev'), Model(id='babbage', created=1649358449, object='model', owned_by='openai'), Model(id='babbage-search-query', created=1651172509, object='model', owned_by='openai-dev'), Model(id='text-babbage-001', created=1649364043, object='model', owned_by='openai'), Model(id='text-similarity-davinci-001', created=1651172505, object='model', owned_by='openai-dev'), Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'), Model(id='davinci-similarity', created=1651172509, object=

In [ ]:
# uploading file to openai API
client = OpenAI(
    organization = "add org",
)
client.files.create(
    file=open("/content/drive/MyDrive/Coterm/CS224V/finetuning_data.jsonl", "rb"),
    purpose="fine-tune"
  )

FileObject(id='file-JZWystgts0ioG1VajLieUnav', bytes=1634425, created_at=1699868743, filename='finetuning_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
# creating finetuned model
client = OpenAI(
    organization = "add org",
)

client.fine_tuning.jobs.create(
  training_file="file-JZWystgts0ioG1VajLieUnav",
  model="gpt-3.5-turbo",
  hyperparameters={
      'n_epochs': 1
  }
)

FineTuningJob(id='ftjob-EyPcOy4q5tI6Zpfx50i0yEJH', created_at=1699872709, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-17hoOBeSUF5UjIcSoCjHZbtK', result_files=[], status='validating_files', trained_tokens=None, training_file='file-JZWystgts0ioG1VajLieUnav', validation_file=None)

In [ ]:
# check status on fine-tune
client = OpenAI(
    organization = "org-17hoOBeSUF5UjIcSoCjHZbtK",
)

# client.fine_tuning.jobs.retrieve("ftjob-dk6X5FZI4oXPUfNHLgEfIMcM")
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-EyPcOy4q5tI6Zpfx50i0yEJH", limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-DKQYTFZNx2IxCVX3h79L3VOw', created_at=1699874628, level='info', message='Step 1001/1079: training loss=0.00', object='fine_tuning.job.event', data={'step': 1001, 'train_loss': 2.119276274470394e-07, 'train_mean_token_accuracy': 1.0}, type='metrics'), FineTuningJobEvent(id='ftevent-DMg9G37BzraEBdLgzODhXOSS', created_at=1699874447, level='info', message='Step 901/1079: training loss=0.41', object='fine_tuning.job.event', data={'step': 901, 'train_loss': 0.4106511175632477, 'train_mean_token_accuracy': 0.978723406791687}, type='metrics'), FineTuningJobEvent(id='ftevent-57Z8CBlULW7kLffxVWs9pHcA', created_at=1699874268, level='info', message='Step 801/1079: training loss=0.60', object='fine_tuning.job.event', data={'step': 801, 'train_loss': 0.5977728962898254, 'train_mean_token_accuracy': 0.9473684430122375}, type='metrics'), FineTuningJobEvent(id='ftevent-iRDd3bwOV5axF98KehKKbmKT', created_at=1699874091, level='info',

### Using OpenAI Finetuned Model

\
    \nSo I will, I will start testing on the development system next week <Commissive> and then, we should be able to get it onto the production system, \
    I think, within um one or two months. <Assertive> But the open question is, we need to define some meters somewhere so. kind of need to agree on where we're putting that. Is this the approach temperature? \
    Is it the dp? is it, is it the Software C freezer monitoring, where do we want to start? Or do we just want to start with one thing to check it out? <Directive>\
    \n\n\
    So the way it works is, we will, Software A will not generate work orders. Software A will send me the data to CMMS and \
    then in CMMS you can define when the dp is more than two for X amount of time generate a work order or service request, one of the two. So I guess that's also open question, \
    do we want to generate work orders or service requests? where a service request is a request that comes in, that needs to be verified by a person before it becomes a work order. \
    We have the option to generate both basically.

In [ ]:
from openai import OpenAI
client = OpenAI(
    organization = "org-17hoOBeSUF5UjIcSoCjHZbtK",
)

row_example = finetuning_df.sample(n=1)
# message prompt with one example
message_prompt = [
    {"role": "system", "content": "You are a chatbot that identifies speech acts within a sequence. You cannot give consecutive utterances the same speech act. Instead they should just have one speech act token at the end of the consecutive utterances.\n"},
    {"role": "user", "content": "and then you know give out warning, but not actually generate any kind of request or work order so that they get used to the integration between Software A and CMMS. And if we want to cut in a new maintenance process that needs to be you know new procedure needs to be written up, you know all the implication needs to be uh addressed, because Company P is saying that if you do that to me with you know, changing the current process, the costs might go up, might go down, you know. They got to work out all that during internal you know interaction between you know the procurement department, the technician out here and the the manpower resource, you know. I had a, I had a discussion with M3 and he really you know, open up my eyes, on the current maintenance protocol. So um i’m definitely gonna be very mindful of the impact of what CBM is gonna do to the current process."},
    {"role": "assistant", "content": "and then you know give out warning, but not actually generate any kind of request or work order so that they get used to the integration between Software A and CMMS. <Directive> And if we want to cut in a new maintenance process that needs to be you know new procedure needs to be written up, you know all the implication needs to be uh addressed, because Company P is saying that if you do that to me with you know, changing the current process, the costs might go up, might go down, you know. They got to work out all that during internal you know interaction between you know the procurement department, the technician out here and the the manpower resource, you know. I had a, I had a discussion with M3 and he really you know, open up my eyes, on the current maintenance protocol. <Assertive> So um i’m definitely gonna be very mindful of the impact of what CBM is gonna do to the current process. <Commissive>"},
    {"role": "user", "content": "yeah yeah no um so yeah I want to use this meeting to kind of talk about you know Company Z. Um A0 and I had a post discussion after our meeting on Wednesday with their presentation. Uh I think A0, I both agree that we should set up a pilot and and do a parallel comparison for the building Omega equipment that's already on wireless on the Company R system. And put Company Z system on it, you know just to kind of compare. Have all the data go thru the Software A, you know and then we'll look at it ourselves, you know so."}



    # {"role": "assistant", "content": "So I will, I will start testing on the development system next week <Commissive> and then, we should be able to get it onto the production system, \
    # I think, within um one or two months. <Assertive> But the open question is, we need to define some meters somewhere so. kind of need to agree on where we're putting that. Is this the approach temperature? \
    # Is it the dp? is it, is it the Software C freezer monitoring, where do we want to start? Or do we just want to start with one thing to check it out? <Directive>"}
]


response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal::8KPN3hQj",
  messages=message_prompt
)
print(response.choices[0].message.content)
latest_response = response.choices[-1].message.content
print(latest_response)

yeah yeah no um so yeah I want to use this meeting to kind of talk about you know Company Z. Um A0 and I had a post discussion after our meeting on Wednesday with their presentation. Uh I think A0, I both agree that we should set up a pilot and and do a parallel comparison for the building Omega equipment that's already on wireless on the Company R system. And put Company Z system on it, you know just to kind of compare. Have all the data go thru the Software A, you know and then we'll look at it ourselves, you know so. <Directive>
yeah yeah no um so yeah I want to use this meeting to kind of talk about you know Company Z. Um A0 and I had a post discussion after our meeting on Wednesday with their presentation. Uh I think A0, I both agree that we should set up a pilot and and do a parallel comparison for the building Omega equipment that's already on wireless on the Company R system. And put Company Z system on it, you know just to kind of compare. Have all the data go thru the Softwar

In [ ]:
print(response)

ChatCompletion(id='chatcmpl-8KPT28QBWJhzDkzcvU4YY3fTIiJCu', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="The equipment wise they do do some kind of PMs but you're talking about maybe the process utility right? They’re they probably don't do much maintenance. <Assertive> ", role='assistant', function_call=None, tool_calls=None))], created=1699875148, model='ft:gpt-3.5-turbo-0613:personal::8KPN3hQj', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=37, prompt_tokens=230, total_tokens=267))


### Preprocess Validation Dataset

In [ ]:
transcript_file_path = '/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/'
valid_df = pd.read_csv(transcript_file_path + '09.10mtg-Table 1.csv')
valid_df.drop(columns = valid_df.columns[-1])

gold_file_path = '/content/drive/MyDrive/Coterm/CS224V/Bi-Weekly-Labeled/'
gold_valid_df = pd.read_csv(gold_file_path + '9.10mtg.csv')
gold_valid_df = gold_valid_df.iloc[:, :5]
gold_valid_df.fillna("NA", inplace=True)

gold_valid_df.tail(10)

,turn,start time,speaker,utterance,speech act
536,443.0,57:27:00,UK,Alright bye,Assertive
537,444.0,57:27:00,E0,Have a nice vacation L0,Expressive
538,445.0,57:28:00,A1,Yea have a nice vacation,Expressive
539,446.0,57:30:00,A0,Yea enjoy,Expressive
540,447.0,57:31:00,L0,bye.,Expressive
541,448.0,57:32:00,end,\n,\n
542,449.0,NA,NA,NA,NA
543,449.0,NA,NA,NA,NA
544,NA,NA,NA,NA,NA
545,NA,NA,NA,NA,NA


In [ ]:
valid_df.tail(10)

,start time,turn,speaker,speaker turn,Unnamed: 4
438,2:9:22,439,E0,okay,NaN
439,2:9:23,440,L0,Alright.,NaN
440,2:9:24,441,A0,okay.,NaN
441,2:9:27:00.000,442,E0,alright see you about a month then.,NaN
442,2:9:27:00.000,443,UK,Alright bye,NaN
443,2:9:27:00.000,444,E0,Have a nice vacation L0,NaN
444,2:9:28,445,A1,Yea have a nice vacation,NaN
445,2:9:30,446,A0,Yea enjoy,NaN
446,2:9:31:00.000,447,L0,bye.,NaN
447,2:9:32,448,end,NaN,NaN


In [ ]:
def concatenate_utterance_speech_act(group):
  last_speech_act = None
  concat_str = ""
  for index in reversed(range(len(group))):
    row = group.iloc[index]
    prefix_str = row['utterance'] + ' '
    if row['speech act'] != last_speech_act:
      last_speech_act = row['speech act']
      prefix_str += '<' + row['speech act'] + '> '
    concat_str = prefix_str + concat_str

  return concat_str

valid_gold_df = gold_valid_df.groupby(['turn', 'speaker']).apply(concatenate_utterance_speech_act).reset_index(name='speaker turn gold')

valid_gold_df.tail(10)

,turn,speaker,speaker turn gold
440,441.0,A0,okay. <Assertive>
441,442.0,E0,alright see you about a month then. <Assertive>
442,443.0,UK,Alright bye <Assertive>
443,444.0,E0,Have a nice vacation L0 <Expressive>
444,445.0,A1,Yea have a nice vacation <Expressive>
445,446.0,A0,Yea enjoy <Expressive>
446,447.0,L0,bye. <Expressive>
447,448.0,end,\n <\n>
448,449.0,NA,NA NA <NA>
449,NA,NA,NA NA <NA>


In [ ]:
valid_gold_df = valid_gold_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
valid_gold_df['speaker turn'] = valid_df['speaker turn']
valid_gold_df.head(10)

,turn,speaker,speaker turn gold,speaker turn
0,1.0,NA,"[silence, view L0’s lock screen, then his emai...","[silence, view L0’s lock screen, then his emails]"
1,2.0,A1,"guys, I can see we're still waiting. We’re sti...","guys, I can see we're still waiting. We’re sti..."
2,3.0,L0,sure <Assertive>,sure
3,4.0,A1,in two minutes. <Commissive>,in two minutes.
4,5.0,NA,[silence] [eventually L0 stops sharing his scr...,[silence] [eventually L0 stops sharing his scr...
5,6.0,A1,[sighs or lets out air] ok. So L0 uh there’s o...,[sighs or lets out air] ok. So L0 uh there’s o...
6,7.0,L0,Oh <Assertive>,Oh
7,8.0,A1,"So I will, I will start testing on the develop...","So I will, I will start testing on the develop..."
8,9.0,L0,yeah um so this this Software A CMMS integrati...,yeah um so this this Software A CMMS integrati...
9,10.0,A1,yeah. So… <Assertive>,yeah. So…


In [ ]:
valid_gold_df.tail(10)

,turn,speaker,speaker turn gold,speaker turn
440,441.0,A0,okay. <Assertive>,okay.
441,442.0,E0,alright see you about a month then. <Assertive>,alright see you about a month then.
442,443.0,UK,Alright bye <Assertive>,Alright bye
443,444.0,E0,Have a nice vacation L0 <Expressive>,Have a nice vacation L0
444,445.0,A1,Yea have a nice vacation <Expressive>,Yea have a nice vacation
445,446.0,A0,Yea enjoy <Expressive>,Yea enjoy
446,447.0,L0,bye. <Expressive>,bye.
447,448.0,end,\n <\n>,NaN
448,449.0,NA,NA NA <NA>,NaN
449,NA,NA,NA NA <NA>,NaN


In [ ]:
valid_gold_df = valid_gold_df[~valid_gold_df.applymap(lambda x: '<NA>' in str(x) or '<N/a>' in str(x)).any(axis=1)]

In [ ]:
valid_gold_df = valid_gold_df.drop(valid_gold_df.index[-1])

In [ ]:
valid_gold_df.head(40)

,turn,speaker,speaker turn gold,speaker turn
1,2.0,A1,"guys, I can see we're still waiting. We’re sti...","guys, I can see we're still waiting. We’re sti..."
2,3.0,L0,sure <Assertive>,sure
3,4.0,A1,in two minutes. <Commissive>,in two minutes.
5,6.0,A1,[sighs or lets out air] ok. So L0 uh there’s o...,[sighs or lets out air] ok. So L0 uh there’s o...
6,7.0,L0,Oh <Assertive>,Oh
7,8.0,A1,"So I will, I will start testing on the develop...","So I will, I will start testing on the develop..."
8,9.0,L0,yeah um so this this Software A CMMS integrati...,yeah um so this this Software A CMMS integrati...
9,10.0,A1,yeah. So… <Assertive>,yeah. So…
10,11.0,L0,So. yeah go ahead <Directive>,So. yeah go ahead
11,12.0,A1,"So the way it works is, we will, Software A wi...","So the way it works is, we will, Software A wi..."


In [ ]:
valid_gold_df.tail(10)

,turn,speaker,speaker turn gold,speaker turn
436,437.0,E0,okay. <Assertive>,okay.
437,438.0,L0,"I mean, unless you guys want to meet you know ...","I mean, unless you guys want to meet you know ..."
438,439.0,E0,okay <Assertive>,okay
439,440.0,L0,Alright. <Assertive>,Alright.
440,441.0,A0,okay. <Assertive>,okay.
441,442.0,E0,alright see you about a month then. <Assertive>,alright see you about a month then.
442,443.0,UK,Alright bye <Assertive>,Alright bye
443,444.0,E0,Have a nice vacation L0 <Expressive>,Have a nice vacation L0
444,445.0,A1,Yea have a nice vacation <Expressive>,Yea have a nice vacation
445,446.0,A0,Yea enjoy <Expressive>,Yea enjoy


In [ ]:
valid_gold_df.to_csv("/content/drive/MyDrive/Coterm/CS224V/gold_validation_biweekly_meetings.csv", index=False)

### Evaluate on Validation Dataset

In [ ]:
validation_df = pd.read_csv("/content/drive/MyDrive/Coterm/CS224V/gold_validation_biweekly_meetings.csv")
validation_df.tail(10)

,turn,speaker,speaker turn gold,speaker turn
394,437.0,E0,okay. <Assertive>,okay.
395,438.0,L0,"I mean, unless you guys want to meet you know ...","I mean, unless you guys want to meet you know ..."
396,439.0,E0,okay <Assertive>,okay
397,440.0,L0,Alright. <Assertive>,Alright.
398,441.0,A0,okay. <Assertive>,okay.
399,442.0,E0,alright see you about a month then. <Assertive>,alright see you about a month then.
400,443.0,UK,Alright bye <Assertive>,Alright bye
401,444.0,E0,Have a nice vacation L0 <Expressive>,Have a nice vacation L0
402,445.0,A1,Yea have a nice vacation <Expressive>,Yea have a nice vacation
403,446.0,A0,Yea enjoy <Expressive>,Yea enjoy


In [ ]:
from openai import OpenAI
client = OpenAI(
    organization = "org-17hoOBeSUF5UjIcSoCjHZbtK",
)

validation_examples = []
system_prompt = {"role": "system",
               "content": "You are a chatbot that identifies speech acts within a sequence."}

for index, row in validation_df.iterrows():
    row_example = finetuning_df.sample(n=1)
    combined_sequence = []
    input_sequence = {
        "role": "user",
        "content": " Your options for the speech acts are Commissive, Assertive, Directive, Expressive, or Declaration. Here are the definitions for each speech act: \n\
Commissive: Speaker utters U to commit to a future action or future state.\n\
Directive: Speaker attempts to get the Hearer to do something or bring about a future state by uttering U.\n\
Expressive: Speaker expresses a psychological state about specific affairs.\n\
Declaration: Upon speaker uttering U the propositional content of U becomes reality.\n\
Assertive: Speaker utters U to describe an existing state of affairs.\n\
Identify and label the shifts in sentiment as they occur by inserting the speech act in <> in between the shifts in speech act. Only write the original input text with the speech act tokens inserted.\n\
Example Input: " + row_example['speaker turn'].iloc[0] + '\nExample Output: ' + row_example['speaker turn gold'].iloc[0] + "\nInput: " + row['speaker turn'] + "\nOutput: "

    }
    # if input_sequence["content"] is None or input_sequence["content"] == '':
    #   print("missing speaker turn: ", row['speaker turn'])
    #   print("missing speaker turn at row: ", index)

    output_sequence = {
        "role": "assistant",
        "content": row['speaker turn gold']
    }
    # if output_sequence["content"] is None or output_sequence["content"] == '':
    #   print("missing speaker turn gold: ", row['speaker turn'])
    #   print("missing speaker turn gold at row: ", index)

    combined_sequence.append(system_prompt)
    combined_sequence.append(input_sequence)
    combined_sequence.append(output_sequence)
    message = {"messages": combined_sequence}
    validation_examples.append(message)


In [ ]:
print(validation_examples)

[{'messages': [{'role': 'system', 'content': 'You are a chatbot that identifies speech acts within a sequence.'}, {'role': 'user', 'content': " Your options for the speech acts are Commissive, Assertive, Directive, Expressive, or Declaration. Here are the definitions for each speech act: \nCommissive: Speaker utters U to commit to a future action or future state.\nDirective: Speaker attempts to get the Hearer to do something or bring about a future state by uttering U.\nExpressive: Speaker expresses a psychological state about specific affairs.\nDeclaration: Upon speaker uttering U the propositional content of U becomes reality.\nAssertive: Speaker utters U to describe an existing state of affairs.\nIdentify and label the shifts in sentiment as they occur by inserting the speech act in <> in between the shifts in speech act. Only write the original input text with the speech act tokens inserted.\nExample Input: right there right yeah right there.\nExample Output: right there right yeah

### DEEPAND preprocessing data

In [ ]:
# transcript_file_path = '/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/'
# raw_df = pd.read_csv(transcript_file_path + '08.13mtg-Table 1.csv')
# raw_df1 = pd.read_csv(transcript_file_path + '08.27mtg-Table 1.csv')
# raw_df2 = pd.read_csv(transcript_file_path + '07.30mtg-Table 1.csv')
# raw_df3 = pd.read_csv(transcript_file_path + '07.16mtg-Table 1.csv')
# raw_df_concat = pd.concat([raw_df, raw_df1, raw_df2, raw_df3], axis=0, ignore_index=True)
# # df.fillna('NA', inplace=True)
# # raw_df_concat = raw_df_concat.dropna(subset=['speaker'])
# raw_df_concat = raw_df_concat.drop(raw_df_concat.columns[-1], axis=1)

# print(len(raw_df))
# print(len(raw_df1))
# print(len(raw_df2))
# print(len(raw_df3))

325
277
283
224


In [ ]:
### Generate/preprocess the gold dataset
gold_transcript_file_path = '/content/drive/MyDrive/Coterm/CS224V/Bi-Weekly-Labeled/'
# gold_df = pd.read_csv(gold_transcript_file_path)

df = pd.read_csv(gold_transcript_file_path + '8.13mtg.csv')
df1 = pd.read_csv(gold_transcript_file_path + '8.27mtg.csv')
df2 = pd.read_csv(gold_transcript_file_path + '7.30mtg.csv')
df3 = pd.read_csv(gold_transcript_file_path + '7.16mtg.csv')

# gold_df = pd.concat([df, df1, df2, df3], axis=0, ignore_index=True)

df = df.iloc[:, :6]
df.fillna("NA", inplace=True)
# df = df.dropna(subset=['speaker', 'speech act'])

df1 = df1.iloc[:, :6]
df1.fillna("NA", inplace=True)
# df1 = df1.dropna(subset=['speaker', 'speech act'])

df2 = df2.iloc[:, :6]
df2.fillna("NA", inplace=True)
# df2 = df2.dropna(subset=['speaker', 'speech act'])

df3 = df3.iloc[:, :6]
df3.fillna("NA", inplace=True)
# df3 = df3.dropna(subset=['speaker', 'speech act'])

print(len(df))
print(len(df1))
print(len(df2))
print(len(df3))
df.head(10)

416
363
386
362


,turn,start time,speaker,utterance,speech act,deepand
0,1.0,0:04,L0,No problem. [to me about recording],NA,NA
1,2.0,0:05,NA,[silence],NA,NA
2,3.0,2:08,A2,[overheard phone conversation] don’t come back...,NA,NA
3,3.0,2:35,A2,[gives thumbs up],NA,NA
4,4.0,2:36,L0,"Hey A2, how’s it going?",Directive,Describe
5,5.0,2:38,NA,[silence],NA,NA
6,6.0,2:49,L0,"Hi M1 did you see uh M0 today? Is he ah, do yo...",Directive,Describe
7,7.0,2:57,M1,"Um, I’m not on site today, but he has sent an ...",Assertive,Describe
8,8.0,3:03,L0,Ok All right.,Assertive,Describe
9,8.0,3:03,L0,I’ll give him a few more minutes see if M0 can...,Commissive,Negotiate


In [ ]:
def concatenate_utterance_deepand(group):
  last_deepand = None
  concat_str = ""
  for index in reversed(range(len(group))):
    row = group.iloc[index]
    prefix_str = row['utterance'] + ' '
    if row['deepand'] != last_deepand:
      last_deepand = row['deepand']
      prefix_str += '<' + row['deepand'] + '> '
    concat_str = prefix_str + concat_str

  return concat_str

def concatenate_utterance(group):
  return ' '.join(group['utterance'])

grouped_df = df.groupby(['turn', 'speaker', 'speech act']).apply(concatenate_utterance_deepand).reset_index(name='speaker turn gold')
input_df = df.groupby(['turn', 'speaker', 'speech act']).apply(concatenate_utterance).reset_index(name='speaker turn')
grouped_df = grouped_df.drop(grouped_df.index[-1])
input_df = input_df.drop(input_df.index[-1])

grouped_df1 = df1.groupby(['turn', 'speaker', 'speech act']).apply(concatenate_utterance_deepand).reset_index(name='speaker turn gold')
input_df1 = df1.groupby(['turn', 'speaker', 'speech act']).apply(concatenate_utterance).reset_index(name='speaker turn')
grouped_df1 = grouped_df1.drop(grouped_df1.index[-1])
input_df1 = input_df1.drop(input_df1.index[-1])

grouped_df2 = df2.groupby(['turn', 'speaker', 'speech act']).apply(concatenate_utterance_deepand).reset_index(name='speaker turn gold')
input_df2 = df2.groupby(['turn', 'speaker', 'speech act']).apply(concatenate_utterance).reset_index(name='speaker turn')
grouped_df2 = grouped_df2.drop(grouped_df2.index[-1])
input_df2 = input_df2.drop(input_df2.index[-1])

grouped_df3 = df3.groupby(['turn', 'speaker', 'speech act']).apply(concatenate_utterance_deepand).reset_index(name='speaker turn gold')
input_df3 = df3.groupby(['turn', 'speaker', 'speech act']).apply(concatenate_utterance).reset_index(name='speaker turn')
grouped_df3 = grouped_df3.drop(grouped_df3.index[-1])
input_df3 = input_df3.drop(input_df3.index[-1])

gold_df = pd.concat([grouped_df, grouped_df1, grouped_df2, grouped_df3], axis=0, ignore_index=True)
gold_input_df = pd.concat([input_df, input_df1, input_df2, input_df3], axis=0, ignore_index=True)

print(len(grouped_df))
print(len(grouped_df1))
print(len(grouped_df2))
print(len(grouped_df3))

359
322
326
275


In [ ]:
gold_input_df.tail(10)

,turn,speaker,speech act,speaker turn
1272,216.0,L0,Expressive,"Okay, thanks."
1273,217.0,A1,Assertive,Nope.
1274,218.0,L0,Directive,"Nope, we’re good. All right, how about you A0?"
1275,219.0,A0,Assertive,Good.
1276,220.0,L0,Assertive,All right.
1277,221.0,A0,Assertive,I know my action items.
1278,222.0,L0,Commissive,So I write up the meeting minutes and go from ...
1279,222.0,L0,Expressive,Thanks guys
1280,223.0,A0,Assertive,Cool
1281,224.0,M1,Expressive,Thank you.


In [ ]:
gold_df.tail(10)

,turn,speaker,speech act,speaker turn gold
1272,216.0,L0,Expressive,"Okay, thanks. <?>"
1273,217.0,A1,Assertive,Nope. <Negotiate>
1274,218.0,L0,Directive,"Nope, we’re good. All right, how about you A0?..."
1275,219.0,A0,Assertive,Good. <Negotiate>
1276,220.0,L0,Assertive,All right. <Negotiate>
1277,221.0,A0,Assertive,I know my action items. <Negotiate>
1278,222.0,L0,Commissive,So I write up the meeting minutes and go from ...
1279,222.0,L0,Expressive,Thanks guys <?>
1280,223.0,A0,Assertive,Cool <?>
1281,224.0,M1,Expressive,Thank you. <?>


In [ ]:
gold_df = gold_df.reset_index(drop=True)
gold_input_df = gold_input_df.reset_index(drop=True)
gold_df['speaker turn'] = gold_input_df['speaker turn']
# drop the na rows
# df = df.dropna(subset=['speaker', 'speech act'])
gold_df.tail(10)

,turn,speaker,speech act,speaker turn gold,speaker turn
1272,216.0,L0,Expressive,"Okay, thanks. <?>","Okay, thanks."
1273,217.0,A1,Assertive,Nope. <Negotiate>,Nope.
1274,218.0,L0,Directive,"Nope, we’re good. All right, how about you A0?...","Nope, we’re good. All right, how about you A0?"
1275,219.0,A0,Assertive,Good. <Negotiate>,Good.
1276,220.0,L0,Assertive,All right. <Negotiate>,All right.
1277,221.0,A0,Assertive,I know my action items. <Negotiate>,I know my action items.
1278,222.0,L0,Commissive,So I write up the meeting minutes and go from ...,So I write up the meeting minutes and go from ...
1279,222.0,L0,Expressive,Thanks guys <?>,Thanks guys
1280,223.0,A0,Assertive,Cool <?>,Cool
1281,224.0,M1,Expressive,Thank you. <?>,Thank you.


In [ ]:
gold_df.head(10)

,turn,speaker,speech act,speaker turn gold,speaker turn
0,1.0,L0,NA,No problem. [to me about recording] <NA>,No problem. [to me about recording]
1,2.0,NA,NA,[silence] <NA>,[silence]
2,3.0,A2,NA,[overheard phone conversation] don’t come back...,[overheard phone conversation] don’t come back...
3,4.0,L0,Directive,"Hey A2, how’s it going? <Describe>","Hey A2, how’s it going?"
4,5.0,NA,NA,[silence] <NA>,[silence]
5,6.0,L0,Directive,"Hi M1 did you see uh M0 today? Is he ah, do yo...","Hi M1 did you see uh M0 today? Is he ah, do yo..."
6,7.0,M1,Assertive,"Um, I’m not on site today, but he has sent an ...","Um, I’m not on site today, but he has sent an ..."
7,8.0,L0,Assertive,Ok All right. <Describe>,Ok All right.
8,8.0,L0,Commissive,I’ll give him a few more minutes see if M0 can...,I’ll give him a few more minutes see if M0 can...
9,9.0,NA,NA,[silence] <NA>,[silence]


In [ ]:
gold_df = gold_df[~gold_df.applymap(lambda x: '<NA>' in str(x) or '<N/a>' in str(x) or '<?>' in str(x)).any(axis=1)]
gold_df.head(10)

,turn,speaker,speech act,speaker turn gold,speaker turn
3,4.0,L0,Directive,"Hey A2, how’s it going? <Describe>","Hey A2, how’s it going?"
5,6.0,L0,Directive,"Hi M1 did you see uh M0 today? Is he ah, do yo...","Hi M1 did you see uh M0 today? Is he ah, do yo..."
6,7.0,M1,Assertive,"Um, I’m not on site today, but he has sent an ...","Um, I’m not on site today, but he has sent an ..."
7,8.0,L0,Assertive,Ok All right. <Describe>,Ok All right.
8,8.0,L0,Commissive,I’ll give him a few more minutes see if M0 can...,I’ll give him a few more minutes see if M0 can...
10,10.0,E0,Directive,L0 are you still are we still waiting for some...,L0 are you still are we still waiting for some...
11,11.0,L0,Assertive,yeah I'm waiting for M0 and A1. <Describe>,yeah I'm waiting for M0 and A1.
12,12.0,E0,Directive,And then we're recording this. <Describe>,And then we're recording this.
13,13.0,L0,Assertive,yeah we're. we're recording these <Describe>,yeah we're. we're recording these
15,15.0,L0,Assertive,But if they can’t join i'm thinking about canc...,But if they can’t join i'm thinking about canc...


In [ ]:
gold_df.to_csv('/content/drive/MyDrive/Coterm/CS224V/deepand_gold_train_meetings.csv', index=False)

In [ ]:
deepand_finetuning_df = pd.read_csv('/content/drive/MyDrive/Coterm/CS224V/deepand_gold_train_meetings.csv')
deepand_finetuning_df.head(10)

,turn,speaker,speech act,speaker turn gold,speaker turn
0,4.0,L0,Directive,"Hey A2, how’s it going? <Describe>","Hey A2, how’s it going?"
1,6.0,L0,Directive,"Hi M1 did you see uh M0 today? Is he ah, do yo...","Hi M1 did you see uh M0 today? Is he ah, do yo..."
2,7.0,M1,Assertive,"Um, I’m not on site today, but he has sent an ...","Um, I’m not on site today, but he has sent an ..."
3,8.0,L0,Assertive,Ok All right. <Describe>,Ok All right.
4,8.0,L0,Commissive,I’ll give him a few more minutes see if M0 can...,I’ll give him a few more minutes see if M0 can...
5,10.0,E0,Directive,L0 are you still are we still waiting for some...,L0 are you still are we still waiting for some...
6,11.0,L0,Assertive,yeah I'm waiting for M0 and A1. <Describe>,yeah I'm waiting for M0 and A1.
7,12.0,E0,Directive,And then we're recording this. <Describe>,And then we're recording this.
8,13.0,L0,Assertive,yeah we're. we're recording these <Describe>,yeah we're. we're recording these
9,15.0,L0,Assertive,But if they can’t join i'm thinking about canc...,But if they can’t join i'm thinking about canc...


### OpenAI finetune

### Load data from our new csv

In [ ]:
preprocessed_df = pd.read_csv('/content/drive/My Drive/Coterm/CS224V/9.10mtg_preprocessed.csv')
# preprocessed_df.head(5)
preprocessed_df.tail(5)

,start time,turn,speaker,speaker turn,speaker turn gold
412,2:9:27:00.000,443,UK,Alright bye,Alright bye <Assertive>
413,2:9:27:00.000,444,E0,Have a nice vacation L0,Have a nice vacation L0 <Expressive>
414,2:9:28,445,A1,Yea have a nice vacation,Yea have a nice vacation <Expressive>
415,2:9:30,446,A0,Yea enjoy,Yea enjoy <Expressive>
416,2:9:31:00.000,447,L0,bye.,bye. <Expressive>


### Model training

In [ ]:
!pip install transformers

NotImplementedError: ignored

In [ ]:
!pip install datasets

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install torch

In [ ]:
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Load your dataset
# df = pd.read_csv('your_data.csv')

# Let's assume df is preprocessed and has the columns 'user_turn' and 'processed_speech'
# For the sake of example, here's a dummy DataFrame creation:
# data = {
#     'user_turn': ["Okay, I'll check on that and get back to you. Can you send me the files? Also, meeting on Tuesday works for me."],
#     'processed_speech': ["Okay, I'll check on that and get back to you <commissive> Can you send me the files? <directive> Also, meeting on Tuesday works for me <assertive>"]
# }
# df = pd.DataFrame(data)

print(preprocessed_df)

# Convert DataFrame to a Hugging Face dataset
dataset = Dataset.from_pandas(preprocessed_df)
print(dataset)

# Define the format function
def format_dataset(example):
    example['input_text'] = example['speaker turn']
    example['target_text'] = example['speaker turn gold']
    return example

# Apply the format function
dataset = dataset.map(format_dataset)
print(dataset)

# Tokenize
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def tokenize_function(examples):
    model_inputs = tokenizer(examples['input_text'], max_length=512, truncation=True, padding=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target_text'], max_length=512, truncation=True, padding=True)

    # labels = labels['input_ids']
    labels = labels.input_ids
    labels[labels == tokenizer.pad_token_id] = -100

    model_inputs['labels'] = labels
    return model_inputs

# Tokenize all the data
tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(tokenized_datasets)
print(tokenized_datasets[0]['labels'])

        start time  turn speaker  \
0          4:54:00     2      A1   
1          5:02:00     3      L0   
2          5:02:00     4      A1   
3          5:48:00     6      A1   
4          6:09:00     7      L0   
..             ...   ...     ...   
412  2:9:27:00.000   443      UK   
413  2:9:27:00.000   444      E0   
414         2:9:28   445      A1   
415         2:9:30   446      A0   
416  2:9:31:00.000   447      L0   

                                          speaker turn  \
0    guys, I can see we're still waiting. We’re sti...   
1                                                 sure   
2                                      in two minutes.   
3    [sighs or lets out air] ok. So L0 uh there’s o...   
4                                                   Oh   
..                                                 ...   
412                                        Alright bye   
413                            Have a nice vacation L0   
414                           Yea have a nice

Map:   0%|          | 0/417 [00:00<?, ? examples/s]

Dataset({
    features: ['start time', 'turn', 'speaker', 'speaker turn', 'speaker turn gold', 'input_text', 'target_text'],
    num_rows: 417
})


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/417 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


ArrowInvalid: ignored

In [ ]:
from transformers import T5Tokenizer

# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def tokenize_and_format(dataset):
    # Tokenize the input and target text with appropriate padding and truncation
    tokenized_input = tokenizer(
        dataset['speaker turn'].tolist(),
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

    tokenized_target = tokenizer(
        dataset['speaker turn gold'].tolist(),
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

    # Create attention masks for the target to ignore padding in the loss calculation
    labels_attention_mask = (tokenized_target.input_ids != tokenizer.pad_token_id).int()

    # Replace padding token id's of the target with -100 so they are ignored in the loss computation
    tokenized_target.input_ids[tokenized_target.attention_mask == 0] = -100

    return {
        'input_ids': tokenized_input.input_ids,
        'attention_mask': tokenized_input.attention_mask,
        'labels': tokenized_target.input_ids,
        'decoder_attention_mask': labels_attention_mask
    }

# Apply the function to your dataframe
# Note: You may need to adjust this if using the Hugging Face datasets library
tokenized_data = tokenize_and_format(df)

In [ ]:
print(tokenized_datasets)

Dataset({
    features: ['start time', 'turn', 'speaker', 'speaker turn', 'speaker turn gold', 'input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 417
})


In [ ]:
# Define the model
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Split the dataset into train and validation sets
train_dataset, eval_dataset = tokenized_datasets.train_test_split(test_size=0.1).values()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/My Drive/Coterm/CS224V/',          # output directory
    num_train_epochs=6,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/content/drive/My Drive/Coterm/CS224V/logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)


In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model('/content/drive/My Drive/Coterm/CS224V/')

Epoch,Training Loss,Validation Loss
1,13.801100,13.654494
2,9.354800,7.967120
3,3.777400,1.811806
4,0.997800,0.227566
5,0.264800,0.080442
6,0.084800,0.047388


In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.0473879911005497, 'eval_runtime': 1.7321, 'eval_samples_per_second': 24.248, 'eval_steps_per_second': 3.464, 'epoch': 6.0}


In [ ]:
print(eval_dataset)
print(eval_dataset[:2]['input_text'])
print(eval_dataset[:2]['input_ids'])

Dataset({
    features: ['start time', 'turn', 'speaker', 'speaker turn', 'speaker turn gold', 'input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 42
})
['Yes, yes should be.', "guys, I can see we're still waiting. We’re still waiting for a couple of people so I just get a cup of coffee real quick, be back"]
[[2163, 6, 4273, 225, 36, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:
    model = model.cuda()

# Select some examples from the validation set
example_inputs = eval_dataset[1:6] # ['input_text'][:5]

print(example_inputs)

input_ids = torch.tensor(example_inputs['input_ids']).to(device)
print(input_ids)

attention_mask = torch.tensor(example_inputs['attention_mask']).to(device)

# Generate predictions
outputs = model.generate(input_ids=input_ids,
                         attention_mask=attention_mask,
                         max_length=512)

# Decode the generated ids to texts
decoded_outputs = [tokenizer.decode(output_ids, skip_special_tokens=True) for output_ids in outputs]

# Print the results
for input_text, decoded_output in zip(example_inputs['input_text'], decoded_outputs):
    print(f"Input: {input_text}")
    print(f"Predicted: {decoded_output}")
    print()

{'start time': ['4:54:00', '1:13:02', '1:1:26', '15:45:00', '12:50:00'], 'turn': [2, 237, 157, 66, 38], 'speaker': ['A1', 'A1', 'A1', 'A1', 'L0'], 'speaker turn': ["guys, I can see we're still waiting. We’re still waiting for a couple of people so I just get a cup of coffee real quick, be back", 'So we’ll look at..\xa0', 'that can help us with installation and some someone we can call to ask about vibration questions.', "yeah and it’s. I think it's also important to compare", 'just just kind of identify what do we want to get out of these test, you know and um and then have a conversation with maintenance folks.'], 'speaker turn gold': ["guys, I can see we're still waiting. We’re still waiting for a couple of people <Assertive> so I just get a cup of coffee real quick, be back <Commissive>", 'So we’ll look at..\xa0 <?>', 'that can help us with installation and some someone we can call to ask about vibration questions. <Directive>', "yeah and it’s. I think it's also important to compare

### Different method with prompting

In [ ]:
!pip install openai==0.28

In [ ]:
import os
import openai
openai.organization = "add org"
openai.api_key = "add api key"

prompt = "The following text contains various sentiments (speech acts). Your options for the speech acts are Commissive, Assertive, Directive, Expressive, or Declaration. Identify and label the shifts in sentiment as they occur. Make sure not to leave anything out from the original input text.\n\
\n\nExample Input: So I will, I will start testing on the development system next week and then, we should be able to get it onto the production system, I think, within um one or two months. But the open question is, we need to define some meters somewhere so. kind of need to agree on where we're putting that. Is this the approach temperature? Is it the dp? is it, is it the Software C freezer monitoring, where do we want to start? Or do we just want to start with one thing to check it out?\n\
Example Output: So I will, I will start testing on the development system next week <Commissive> and then, we should be able to get it onto the production system, I think, within um one or two months.: <Assertive> But the open question is, we need to define some meters somewhere so. kind of need to agree on where we're putting that. Is this the approach temperature? Is it the dp? is it, is it the Software C freezer monitoring, where do we want to start? Or do we just want to start with one thing to check it out?: <Directive>\n\n\
Here is the text to label with the speech acts:\n\
Input Text: "

prompt += "ok. So L0 uh there’s one thing I think we should go over in this meeting. So the Software A to CMMS integration, all the file, all the stuff that takes time basically has been approved and hopefully it will be done this week."

prompt += "\"\n\Output: "

print(prompt)

response = openai.Completion.create(
  model="text-davinci-003",  # Replace with gpt-4 model when available.
  prompt=prompt,
  temperature=0.5,
  max_tokens=150,
  n=1,
  stop=None,
  logprobs=None
)

print(response.choices[0].text.strip())

The following text contains various sentiments (speech acts). Your options for the speech acts are Commissive, Assertive, Directive, Expressive, or Declaration. Identify and label the shifts in sentiment as they occur. Make sure not to leave anything out from the original input text.


Example Input: So I will, I will start testing on the development system next week and then, we should be able to get it onto the production system, I think, within um one or two months. But the open question is, we need to define some meters somewhere so. kind of need to agree on where we're putting that. Is this the approach temperature? Is it the dp? is it, is it the Software C freezer monitoring, where do we want to start? Or do we just want to start with one thing to check it out?
Example Output: So I will, I will start testing on the development system next week <Commissive> and then, we should be able to get it onto the production system, I think, within um one or two months.: <Assertive> But the 

In [ ]:
### parse the output

import re

def parse_text(text):
    text_list = []
    tag_list = []

    # Define a regular expression pattern to match the text and tags
    pattern = r'(.+?):\s<(Assertive|Directive)>'

    # Find all matches in the text
    matches = re.findall(pattern, text)

    for match in matches:
        # Append the text and tag to their respective lists
        text_list.append(match[0].strip())
        tag_list.append(match[1])

    return text_list, tag_list

# Example text
example_text = response.choices[0].text.strip()

text_list, tag_list = parse_text(example_text)

# Print the results
print("Text List:")
print(text_list)

print("\nTag List:")
print(tag_list)

Text List:
['ok.', 'So L0 uh there’s one thing I think we should go over in this meeting.', 'So the Software A to CMMS integration, all the file, all the stuff that takes time basically has been approved and hopefully it will be done this week.']

Tag List:
['Assertive', 'Directive', 'Assertive']


### Connect to Together.ai

In [ ]:
!pip install --upgrade together

In [ ]:
!pip install datasets

In [ ]:
import together

together.api_key = "d7b9530f733a31a114a2525b188757e8029b3b3ba86820736713a36b5138bce7"


In [ ]:
%%bash

export TOGETHER_API_KEY=d7b9530f733a31a114a2525b188757e8029b3b3ba86820736713a36b5138bce7
together models info togethercomputer/llama-2-7b

+----------------------+---------------------------------------------------------------------------------------------------------------------------------------------+
| name                 | togethercomputer/llama-2-7b                                                                                                                 |
+----------------------+---------------------------------------------------------------------------------------------------------------------------------------------+
| display_name         | LLaMA-2 (7B)                                                                                                                                |
+----------------------+---------------------------------------------------------------------------------------------------------------------------------------------+
| display_type         | language                                                                                                                                    

In [ ]:
from datasets import Dataset

# dataset = Dataset.from_pandas(preprocessed_df)

# # Define the input and target text columns
# def format_dataset(example):
#     # The input could just be the user turn, since the model will learn to associate utterances with the speech act labels
#     example['input_text'] = example['speaker turn']
#     # The target is the processed speech which concatenates utterances with speech act labels within a turn
#     example['target_text'] = example['speaker turn gold']
#     return example

# # Apply the function to format the dataset
# dataset = dataset.map(format_dataset)
# print(dataset)


# def tokenize_function(examples):
#     model_inputs = tokenizer(examples['input_text'], max_length=512, truncation=True)
#     # Setup the tokenizer for targets
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(examples['target_text'], max_length=512, truncation=True)

#     model_inputs['labels'] = labels['input_ids']
#     return model_inputs


# togethercomputer/llama-2-70b


Map:   0%|          | 0/418 [00:00<?, ? examples/s]

Dataset({
    features: ['start time', 'turn', 'speaker', 'speaker turn', 'speaker turn gold', 'input_text', 'target_text'],
    num_rows: 418
})


In [ ]:
data_list = []

for index, row in preprocessed_df.iterrows():
    # print(row['speaker turn'])
    # print(row['speaker turn gold'])

    # instruction_input_separator = random.choice([":", ": ", "\n", "\n\n", " "])
    # input = sample['input'] if sample['input'] is not None else ""
    # instruction = sample['instruction'] if sample['instruction'] is not None else ""

    # training_sequence = format_to_llama2_chat(
    #     "you are a helpful legal assistant",
    #     [(instruction+instruction_input_separator+input,sample['output'])]
    # )
    # print(sample)

    sequence = "[input]:" + row['speaker turn'] + '[output]:' + row['speaker turn gold']

    data_list.append({
        "text":sequence
    })

print(len(data_list))
print(data_list[0])

417
{'text': "[input]:guys, I can see we're still waiting. We’re still waiting for a couple of people so I just get a cup of coffee real quick, be back[output]:guys, I can see we're still waiting. We’re still waiting for a couple of people <Assertive> so I just get a cup of coffee real quick, be back <Commissive>"}


In [ ]:
print(data_list[3])

{'text': '[input]:[sighs or lets out air] ok. So L0 uh there’s one thing I think we should go over in this meeting. So the Software A to CMMS integration, all the file, all the stuff that takes time basically has been approved and hopefully it will be done this week.[output]:[sighs or lets out air] ok. So L0 uh there’s one thing I think we should go over in this meeting. <Directive> So the Software A to CMMS integration, all the file, all the stuff that takes time basically has been approved and hopefully it will be done this week. <Assertive>'}


In [ ]:
together.Files.save_jsonl(data_list, "/content/drive/My Drive/Coterm/CS224V/9.10mtg_preprocessed.jsonl")

Wrote 417 records to /content/drive/My Drive/Coterm/CS224V/9.10mtg_preprocessed.jsonl


In [ ]:
%%bash
cd drive
cd MyDrive
cd Coterm
cd CS224V
ls

export TOGETHER_API_KEY=d7b9530f733a31a114a2525b188757e8029b3b3ba86820736713a36b5138bce7

together files upload 9.10mtg_preprocessed.jsonl


9.10mtg_preprocessed.csv
9.10mtg_preprocessed.jsonl
Bi-Weekly-Labeled
cbm_biweekly_transcripts
CS 224V Effective Meeting Summaries Progress.docx
CS224V Final Project Proposal.gdoc
CS224v project.ipynb
{
    "filename": "9.10mtg_preprocessed.jsonl",
    "id": "file-73f19b52-e0ca-4816-9387-9714727004f3",
    "object": "file",
    "report_dict": {
        "is_check_passed": true,
        "model_special_tokens": "we are not yet checking end of sentence tokens for this model",
        "file_present": "File found",
        "file_size": "File size 0.0 GB",
        "num_samples": 417
    }
}


Uploading 9.10mtg_preprocessed.jsonl: 100%|██████████| 92.9k/92.9k [00:00<00:00, 146kB/s]


In [ ]:
%%bash
cd drive/MyDrive/Coterm/CS224V


together files check 9.10mtg_preprocessed.jsonl

{
    "is_check_passed": true,
    "model_special_tokens": "we are not yet checking end of sentence tokens for this model",
    "file_present": "File found",
    "file_size": "File size 0.0 GB",
    "num_samples": 417
}


In [ ]:
%%bash

export TOGETHER_API_KEY=d7b9530f733a31a114a2525b188757e8029b3b3ba86820736713a36b5138bce7
echo "y" | together finetune create --training-file file-73f19b52-e0ca-4816-9387-9714727004f3 --model togethercomputer/llama-2-7b --n-epochs 1 --batch-size 8

Job creation details:
{   'batch_size': 8,
    'learning_rate': 1e-05,
    'model': 'togethercomputer/llama-2-7b',
    'n_checkpoints': 1,
    'n_epochs': 1,
    'suffix': None,
    'training_file': 'file-73f19b52-e0ca-4816-9387-9714727004f3',
    'wandb_key': None}

Do you want to submit the job? [y/N]{
    "training_file": "file-73f19b52-e0ca-4816-9387-9714727004f3",
    "validation_file": "",
    "model_output_name": "jlim23@stanford.edu/llama-2-7b-2023-11-06-06-36-47",
    "model_output_path": "s3://together-dev/finetune/653d875edd226b804c17900f/jlim23@stanford.edu/llama-2-7b-2023-11-06-06-36-47/ft-0c2a9b43-aaf4-40a9-a988-603be0d96b47",
    "Suffix": "",
    "model": "togethercomputer/llama-2-7b",
    "n_epochs": 1,
    "n_checkpoints": 1,
    "batch_size": 8,
    "learning_rate": 1e-05,
    "user_id": "653d875edd226b804c17900f",
    "staring_epoch": 0,
    "training_offset": 0,
    "checkspoint_path": "",
    "random_seed": "",
    "created_at": "2023-11-06T06:36:47.851Z",
    "up

In [ ]:
%%bash

export TOGETHER_API_KEY=d7b9530f733a31a114a2525b188757e8029b3b3ba86820736713a36b5138bce7
together finetune list-events ft-0c2a9b43-aaf4-40a9-a988-603be0d96b47

+----+------------------------------------------------------------------------------------------+---------------------------------+----------------------+
|    | Message                                                                                  | Type                            | Hash                 |
+====+==========================================================================================+=================================+======================+
|  0 | Fine tune request created                                                                | JOB_PENDING                     |                      |
+----+------------------------------------------------------------------------------------------+---------------------------------+----------------------+
|  1 | Job started at Sun Nov  5 22:36:54 PST 2023                                              | JOB_START                       | -9105833153480747176 |
+----+----------------------------------------------------------------

In [ ]:
%%bash

export TOGETHER_API_KEY=d7b9530f733a31a114a2525b188757e8029b3b3ba86820736713a36b5138bce7
together finetune retrieve ft-0c2a9b43-aaf4-40a9-a988-603be0d96b47

+--------------------+----------------------------------------------------+
| training_file      | file-73f19b52-e0ca-4816-9387-9714727004f3          |
+--------------------+----------------------------------------------------+
| validation_file    |                                                    |
+--------------------+----------------------------------------------------+
| model_output_name  | jlim23@stanford.edu/llama-2-7b-2023-11-06-06-36-47 |
+--------------------+----------------------------------------------------+
| Suffix             |                                                    |
+--------------------+----------------------------------------------------+
| model              | togethercomputer/llama-2-7b                        |
+--------------------+----------------------------------------------------+
| n_epochs           | 1                                                  |
+--------------------+----------------------------------------------------+
| n_checkpoi

In [ ]:
%%bash

export TOGETHER_API_KEY=d7b9530f733a31a114a2525b188757e8029b3b3ba86820736713a36b5138bce7
together models start jlim23@stanford.edu/llama-2-7b-2023-11-06-06-36-47

{
    "success": true,
    "value": "0dadce5fc30c7bcccbe4eaa102d462f6a7d5edd94cb45078bf4c7ece71c376ac-79accea4806fa1473ef3ec5333cd4702653694950d7255388bfdbe0d8202cb53"
}


In [ ]:
%%bash

export TOGETHER_API_KEY=d7b9530f733a31a114a2525b188757e8029b3b3ba86820736713a36b5138bce7
together models ready jlim23@stanford.edu/llama-2-7b-2023-11-06-06-36-47

[
    {
        "ready": false
    }
]


### Attempt BERT NER

In [ ]:
from transformers import BertTokenizer
import torch

# Multiple lines of input data
input_data_lines = [
    "So I will, I will start testing on the development system next week <Commissive> and then, we should be able to get it onto the production system, I think, within um one or two months. <Assertive>",
    # Add more lines of input data as needed
]

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
speech_act_ids = {"commissive": 1, "assertive": 2, "declaration": 3, "directive": 4, "expressive": 5}

# Prepare training data for each line
train_data = []

for input_data_line in input_data_lines:
    # Tokenize the input data
    tokenized_data = tokenizer(input_data_line, return_tensors="pt")

    # Identify the indices of <Commissive> and <Assertive> tokens
    commissive_index = (tokenized_data["input_ids"][0].index(tokenizer.convert_tokens_to_ids("<Commissive>")), speech_act_ids["commissive"])
    assertive_index = (tokenized_data["input_ids"][0].index(tokenizer.convert_tokens_to_ids("<Assertive>")), speech_act_ids["assertive"])
    declaration_index = (tokenized_data["input_ids"][0].index(tokenizer.convert_tokens_to_ids("<Declaration>")), speech_act_ids["declaration"])
    directive_index = (tokenized_data["input_ids"][0].index(tokenizer.convert_tokens_to_ids("<Directive>")), speech_act_ids["directive"])
    expressive_index = (tokenized_data["input_ids"][0].index(tokenizer.convert_tokens_to_ids("<Expressive>")), speech_act_ids["expressive"])

    sorted_indexes = sort([commissive_index, assertive_index, declaration_index, directive_index, expressive_index])
    # Create labels based on the identified indices
    labels = torch.zeros(len(tokenized_data["input_ids"][0]))
    prev = 0
    for idx in sorted_indexes:
      labels[prev:idx[0]] = idx[1]
      prev = idx[0]

    # Append data to the training set
    train_data.append({"input_ids": tokenized_data["input_ids"][0], "labels": labels})

# Define the number of labels
num_labels = 5  # 1: Commissive, 2: Assertive, , etc.

# Create a custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return {"input_ids": self.data[index]["input_ids"], "labels": self.data[index]["labels"]}

# Create dataset and data loader
train_dataset = CustomDataset(train_data)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)



In [ ]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn as nn

# Model training (adapt as needed)
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3

# Create a learning rate scheduler with warm-up
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


loss_function = nn.CrossEntropyLoss()


In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs, targets = batch
        outputs = model(**inputs, labels=targets)
        loss = outputs.loss

        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}")


model.save_pretrained("/content/drive/My Drive/Coterm/CS224V/pretrained_NER")
tokenizer.save_pretrained("/content/drive/My Drive/Coterm/CS224V/pretrained_NER")


### Naive Meeting Summarization

##### Compile transcripts to be just speaker turns with the utterance in a row

In [ ]:
import pandas as pd

transcript_file_path = '/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/'
raw_df = pd.read_csv(transcript_file_path + '09.10mtg-Table 1.csv')


transcript_str = ''
for index, row in raw_df.iterrows():
  speaker = row['speaker']
  speaker_turn = row['speaker turn']
  transcript_str += str(speaker) + ': ' + str(speaker_turn) + '\n'

print(transcript_str)

##### Naive summarization with raw transcript

In [ ]:
import os
from openai import OpenAI
openai.organization = "add org"
openai.api_key = "add api key"

system_message = "You are a meeting summarizer. Summarize this meeting transcript and split it into sections: what the meeting was about, what speakers committed to doing after the meeting, \
what action items need to be done, and what conversation breakdowns occured. A breakdown is when there is some miscommunication that prevents the meeting from progressing. \
For each topic, provide details and context about that topic. For the action items, add explanations of the items such that someone who is somewhat unfamiliar with the domain could understand. \
Add lots of details, the summary does not have to be super concise."


prompt = 'Transcript:\n' + transcript_str

example_prompt = "Transcript:\n[sample transcript]"

example_summary = "What the meeting was about:\n\
[high level summary]:\n\
1. topic 1...\n\
2. topic 2...\n\
3. ...\n\
What speakers committed to doing after the meeting:\n\
1. A1 is going to do ...\n\
2. ...\n\
Action items that need to be done:\n\
1. action item 1: action item description...\n\
2. action item 2: ...\n\
Conversation breakdowns:\n\
[summary of breakdowns]\n\
1. explanation of breakdown 1.\n\
2. explanation of breakdown 2...\n"

# once you get the highlevel entites that were discussed, add detail about hwat they are about

# print(prompt)

# response = openai.Completion.create(
#   model='gpt-3.5-turbo-1106', # "text-davinci-003",  # Replace with gpt-4 model when available.
#   prompt=prompt,
#   temperature=0.5,
#   max_tokens=150,
#   n=1,
#   stop=None,
#   logprobs=None
# )

# print(response.choices[0].text.strip())

client = OpenAI(
    organization="add org",
    api_key="add api key"
)

response = client.chat.completions.create(
    model="gpt-4-1106-preview", # "gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": example_prompt},
        {"role": "assistant", "content": example_summary},
        {"role": "user", "content": prompt},
    ]
)

# Extract the assistant's reply from the response
assistant_reply = response.choices[0].message.content

# Print or use the assistant's reply
print(assistant_reply)

What the meeting was about:
The meeting revolved around several key topics related to software integration, maintenance processes, sensor data, and a pilot testing of a new system:

1. Software A to CMMS Integration: The integration effort of Software A with the Computerized Maintenance Management System (CMMS) was discussed, including the need for testing on the development system, setting up parameters, deciding where to start, and the implications on Company P's maintenance process.

2. Condition-Based vs. Schedule-Based Maintenance: A deeper conversation took place concerning the shift from schedule-based to condition-based maintenance supported by CMMS, the changes it might introduce to Company P's operations, vendor relations, logistics, budget, and the need for new procedures.

3. Pilot Testing and Machine Learning: The potential for using machine learning to monitor equipment health was considered along with setting up a pilot test to compare Company Z's technology with existin

In [ ]:
from docx import Document

assistant_reply = assistant_reply.replace('*', '').replace('#', '')

doc = Document()
doc.add_paragraph(assistant_reply)
doc.save('/content/drive/MyDrive/Coterm/CS224V/9.10_meeting_summary_GPT4_NAIVE.docx')

### Find breakdowns in structured data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

def get_directive_product_labeled(file_path):
  structured_data = file_path # '/content/drive/MyDrive/Coterm/CS224V/Bi-Weekly-Labeled/9.10mtg.csv'
  structured_df = pd.read_csv(structured_data)

  structured_df = structured_df.iloc[:, :10]

  structured_df['speaker'] = structured_df['speaker'].fillna('NA')


  # Create a new column 'newColumn' based on conditions
  structured_df['directive_product'] = (structured_df['speech act'].str.lower() == 'directive') & (structured_df['pop'].str.lower() == 'product')

  # Convert boolean values to 1s and 0s if needed
  structured_df['directive_product'] = structured_df['directive_product'].astype(int)

  return structured_df

  # Display the DataFrame
  # structured_df.head(20)
  # print(len(structured_df))


For the rows that have directive+product == 1, get context window and determine if breakdown

In [ ]:
# Assuming df is your DataFrame

# Function to concatenate utterances
def concatenate_utterances(index, df):
  # Handle the edge cases where the index is at the start or end of the DataFrame
  start_index = max(index - 8, 0)
  end_index = min(index + 8, len(df))
  # Concatenate the utterances
  return ' '.join(df.loc[start_index:end_index, 'utterance'])

# Function to concatenate utterances
def concatenate_breakdown_context(index, df):
  # Handle the edge cases where the index is at the start or end of the DataFrame
  start_index = max(index - 2, 0)
  end_index = min(index + 2, len(df))
  # Concatenate the utterances
  return ' '.join(df.loc[start_index:end_index, 'utterance'])

# Function to concatenate utterances
def concatenate_context_speakers(index, df):
  # Handle the edge cases where the index is at the start or end of the DataFrame
  start_index = max(index - 2, 0)
  end_index = min(index + 2, len(df))
  # Concatenate the utterances
  return ', '.join(df.loc[start_index:end_index, 'speaker'])


def make_directive_product_windows(structured_df):
  # Create a new column and initialize with empty strings
  structured_df['combined_utterance'] = ''
  structured_df['breakdown_context'] = ''
  structured_df['context_speakers'] = ''

  # Iterate over rows
  for index, row in structured_df.iterrows():
    # Check if 'Directive + Product' is TRUE
    if row['directive_product'] == 1:
      structured_df.at[index, 'combined_utterance'] = concatenate_utterances(index, structured_df)
      structured_df.at[index, 'breakdown_context'] = concatenate_breakdown_context(index, structured_df)
      structured_df.at[index, 'context_speakers'] = concatenate_context_speakers(index, structured_df)

  return structured_df

  # The df now contains a new column 'combined_utterance' with the concatenated utterances
  # structured_df.head(20)

From here we now have a column that contains the context utterance for each row that has directive + product

In [ ]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
# for each row that has directive and product, ask LLM if it contains a breakdown.
import os
from openai import OpenAI
import openai
import numpy as np


def determine_breakdown(structured_df):
  openai.organization = "add org"
  openai.api_key = "add api key"


  client = OpenAI(
      organization="add org",
      api_key="add api key"
  )

  system_message = "You are finding breakdowns in meeting conversations. \
A breakdown in a conversation exists when there is a miscommunication, a need for clarification, or a gap in information that prevents the meeting from progressing smoothly. \
Determine if an utterance contains a breakdown by reading the context conversation, then decide whether the utterance is a breakdown. \
Output a 1 if there is a breakdown, otherwise 0."

  structured_df['breakdown'] = np.nan

  copy_df = structured_df

  for index, row in copy_df.iterrows():
    if row['directive_product'] == 1:
      prompt = "Utterance with potential breakdown: " + row['utterance'] + '\n\n'
      prompt += "Context conversation: " + row['combined_utterance']

      # print(prompt)


      response = client.chat.completions.create(
          model="gpt-4-1106-preview", # "gpt-3.5-turbo-1106",
          messages=[
              {"role": "system", "content": system_message},
              {"role": "user", "content": prompt},
          ]
      )

      # Extract the assistant's reply from the response
      assistant_reply = response.choices[0].message.content
      # print(assistant_reply)
      # print('-----------------------------------------')
      # row['breakdown'] = assistant_reply
      copy_df.at[index, 'breakdown'] = assistant_reply


  # copy_df.head(130)
  copy_df['breakdown'] = copy_df['breakdown'].astype(str)
  return copy_df


In [ ]:
def make_breakdown_windows(copy_df):
  index_with_ones = copy_df[copy_df['breakdown'] == 1].index
  print('index with ones:', len(index_with_ones))
  # print(index_with_ones)

  # Create a list of indices for the rows within the window
  indices_in_window = []
  for index in index_with_ones:
      window_indices = range(max(0, index - 8), min(len(copy_df), index + 9))  # Window of 5 rows, adjusted for boundaries
      indices_in_window.extend(window_indices)

  # Remove duplicates and ensure the indices are within the valid range
  indices_in_window = list(set(filter(lambda x: 0 <= x < len(copy_df), indices_in_window)))

  # Select the rows within the window
  breakdown_df = copy_df.iloc[indices_in_window]

  # Now df_filtered contains a window of 5 rows around each row where 'column_name' has the value 1
  # breakdown_df.head(10)
  return breakdown_df

In [ ]:
def group_breakdown_windows(breakdown_df):
  # Initialize an empty DataFrame to store the grouped rows
  result_df = pd.DataFrame(columns=['turn', 'speaker', 'utterance'])

  current_group = []
  for index, row in breakdown_df.iterrows():
      if not current_group or row['turn'] == current_group[-1]['turn'] or row['turn'] == current_group[-1]['turn'] + 1:
          current_group.append(row)
      else:
          # Concatenate rows in the current group and add to the result DataFrame
          result_row = {
              'turn': current_group[0]['turn'],
              'speaker': ', '.join(list(set([str(r['speaker']) for r in current_group if not str(r['speaker']) == 'nan']))),
              'utterance': ' '.join([r['utterance'] for r in current_group]),
          }
          result_df = result_df.append(result_row, ignore_index=True)
          current_group = [row]

  # Concatenate the last group if it exists
  if current_group:
      result_row = {
          'turn': current_group[0]['turn'],
          'speaker': ', '.join(list(set([r['speaker'] for r in current_group]))),
          'utterance': ' '.join([r['utterance'] for r in current_group]),
      }
      result_df = result_df.append(result_row, ignore_index=True)

  print('length of grouped breakdown windows:', str(len(result_df)))

  return result_df
  # Print the resulting DataFrame
  # result_df.head(20)

In [ ]:
def make_save_breakdown_str(result_df, output_file):
  breakdown_convos = ''
  for ind, row in result_df.iterrows():
    if row['breakdown'] == 1 or row['breakdown'] == '1':
      breakdown_convos += row['context_speakers'] + ': ' + row['breakdown_context'] + '\n'

  print(breakdown_convos)

  with open(output_file, 'w') as txt_file:
    txt_file.write(breakdown_convos)


##### Final function to pass in file path and save the breakdown convos in file

In [ ]:
def find_breakdowns(input_file_path, output_file_path):
  structured_df = get_directive_product_labeled(input_file_path)
  structured_df = make_directive_product_windows(structured_df)
  copy_df = determine_breakdown(structured_df)
  print('to csv')
  copy_df.to_csv('/content/drive/MyDrive/Coterm/CS224V/complex1_preprocessed_7.16.csv', index=False)


find_breakdowns('/content/drive/MyDrive/Coterm/CS224V/Bi-Weekly-Labeled/7.16mtg.csv', '/content/drive/MyDrive/Coterm/CS224V/breakdown_convos_7.16.txt')

to csv


Generate breakdown summary convos

In [ ]:

copy_df = pd.read_csv('/content/drive/MyDrive/Coterm/CS224V/find_breakdowns_our_version_9.10.csv')
copy_df['breakdown'] = copy_df['breakdown'].fillna(0).astype(int).astype(str)

# print(copy_df)
# breakdown_df = make_breakdown_windows(copy_df)
# result_df = group_breakdown_windows(breakdown_df)
make_save_breakdown_str(copy_df, '/content/drive/MyDrive/Coterm/CS224V/mofos_9.10.txt')

A1, L0, A1, L0, A1: [sighs or lets out air] ok. So L0 uh there’s one thing I think we should go over in this meeting. So the Software A to CMMS integration, all the file, all the stuff that takes time basically has been approved and hopefully it will be done this week. Oh So I will, I will start testing on the development system next week and then, we should be able to get it onto the production system, I think, within um one or two months. But the open question is, we need to define some meters somewhere so. kind of need to agree on where we're putting that. Is this the approach temperature? Is it the dp? is it, is it the Software C freezer monitoring, where do we want to start? Or do we just want to start with one thing to check it out? yeah um so this this Software A CMMS integration, we want to use the CMMS to generate the work order right? yeah. So…
L0, A1, L0, A1, L0: Oh So I will, I will start testing on the development system next week and then, we should be able to get it onto

### Evaluate breakdown finder

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

def evaluate_breakdown(gold_file_path, generated_file_path):
  # '/content/drive/MyDrive/Coterm/CS224V/Bi-Weekly-Labeled/9.10mtg.csv'
  gold_structured_df = pd.read_csv(gold_file_path)
  generated_structured_df = pd.read_csv(generated_file_path)

  # for the gold, mark all turns that have one utterance with breakdown true as all breakdown
  gold_structured_df['speaker turn is part of breakdown?'] = gold_structured_df['speaker turn is part of breakdown?'].apply(lambda x: 1 if x == 1 else 0)
  gold_structured_df['speaker turn is part of breakdown?'] = gold_structured_df.groupby('turn')['speaker turn is part of breakdown?'].transform('max')
  gold_structured_df['speaker turn is part of breakdown?'] = gold_structured_df['speaker turn is part of breakdown?'].fillna(0)
  # return gold_structured_df

  # for the generated, mark all turns taht have one utterance with breakdown true as all breakdown
  generated_structured_df['breakdown'] = generated_structured_df['breakdown'].apply(lambda x: 1 if x == 1 else 0)
  generated_structured_df['breakdown'] = generated_structured_df.groupby('turn')['breakdown'].transform('max')
  generated_structured_df['breakdown'] = generated_structured_df['breakdown'].fillna(0)

  # return generated_structured_df

  accuracy = accuracy_score(gold_structured_df['speaker turn is part of breakdown?'], generated_structured_df['breakdown'])
  precision = precision_score(gold_structured_df['speaker turn is part of breakdown?'], generated_structured_df['breakdown'])
  recall = recall_score(gold_structured_df['speaker turn is part of breakdown?'], generated_structured_df['breakdown'])
  f1 = f1_score(gold_structured_df['speaker turn is part of breakdown?'], generated_structured_df['breakdown'])

  print('accuracy:', accuracy)
  print('precision:', precision)
  print('recall:', recall)
  print('f1:', f1)

generated_structured_df = evaluate_breakdown('/content/drive/MyDrive/Coterm/CS224V/Bi-Weekly-Labeled/7.16mtg.csv', '/content/drive/MyDrive/Coterm/CS224V/complex1_preprocessed_7.16.csv')
# gold_structured_df.iloc[160:180]
# generated_structured_df.tail(20)

accuracy: 0.9558011049723757
precision: 0.8888888888888888
recall: 0.8615384615384616
f1: 0.8750000000000001


Calculate weighted scores for all transcripts

In [ ]:
import numpy as np

file_paths = ['/content/drive/MyDrive/Coterm/CS224V/complex1_preprocessed_9.10.csv', '/content/drive/MyDrive/Coterm/CS224V/complex1_preprocessed_8.27.csv', '/content/drive/MyDrive/Coterm/CS224V/complex1_preprocessed_8.13.csv', '/content/drive/MyDrive/Coterm/CS224V/complex1_preprocessed_7.30.csv', '/content/drive/MyDrive/Coterm/CS224V/complex1_preprocessed_7.16.csv']
num_dir_prod = []
for path in file_paths:
  df = pd.read_csv(path)
  true_count = df['directive_product'].astype(int).sum()

  num_dir_prod.append(true_count)

total = sum(num_dir_prod)
num_dir_prod = num_dir_prod / total
num_dir_prod = np.array(num_dir_prod)

# accuracy:
accuracies = np.array([0.9725274725274725, 0.9008264462809917, 0.9543269230769231, 0.9326424870466321, 0.9558011049723757])
recalls = np.array([0.8205128205128205, 0.8214285714285714, 0.9032258064516129, 0.896551724137931, 0.8615384615384616])
precisions = np.array([0.8, 0.42592592592592593, 0.6363636363636364, 0.7222222222222222, 0.8888888888888888])
f1s = np.array([0.810126582278481, 0.5609756097560976, 0.7466666666666666, 0.7999999999999999, 0.8750000000000001])

accuracy = num_dir_prod.dot(accuracies)
recall = num_dir_prod.dot(recalls)
precision = num_dir_prod.dot(precisions)
f1 = num_dir_prod.dot(f1s)

print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)



accuracy: 0.9412944095728806
precision: 0.7001218212273488
recall: 0.8626600929939088
f1: 0.7630011356566717


### Complex Meeting Summarization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##### Load in structured data

In [ ]:
import pandas as pd

transcript_file_path = '/content/drive/MyDrive/Coterm/CS224V/Bi-Weekly-Labeled/'
structured_df = pd.read_csv(transcript_file_path + '9.10mtg.csv')

structured_df.head(10)

,turn,start time,speaker,utterance,speech act,deepand,rrs,pop,ffb,conversation,Directive + Product,is new turn?,item,item property(ies),action,relation between item and action,speaker turn is part of breakdown?,Unnamed: 17,Dir + Process,Unnamed: 19
0,1.0,0:00,NaN,"[silence, view L0’s lock screen, then his emails]",N/a,N/a,N/a,N/a,N/a,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
1,2.0,4:54,A1,"guys, I can see we're still waiting. We’re sti...",Assertive,Describe,statement,Process,form,orientation,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
2,2.0,4:54,A1,"so I just get a cup of coffee real quick, be back",Commissive,Explain,statement,Process,form,action,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
3,3.0,5:02,L0,sure,Assertive,?,response,Process,form,action,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
4,4.0,5:02,A1,in two minutes.,Commissive,Describe,statement,Process,behavior,action,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
5,5.0,5:03,NaN,[silence] [eventually L0 stops sharing his scr...,N/a,N/a,N/a,N/a,N/a,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
6,6.0,5:48,A1,[sighs or lets out air] ok. So L0 uh there’s o...,Directive,Negotiate,statement,Process,function,possibility,False,0.0,NaN,NaN,go over in the meeting,NaN,0.0,NaN,True,NaN
7,6.0,5:48,A1,"So the Software A to CMMS integration, all the...",Assertive,Describe,statement,Process,behavior,orientation,False,0.0,NaN,NaN,Software A to CMMS integration approved; done ...,NaN,NaN,NaN,False,NaN
8,7.0,6:09,L0,Oh,Assertive,N/a,response,N/a,N/a,orientation,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
9,8.0,6:09,A1,"So I will, I will start testing on the develop...",Commissive,Negotiate,statement,Process,form,action,False,0.0,development system,NaN,start testing next week,start testing on the development system next week,NaN,NaN,False,NaN


##### Get breakdonw rows and rows of context for them

In [ ]:
##V2
# Find rows with breakdowns

# Assuming df is your DataFrame and 'column_name' is the column you're interested in
# copy_df_no_nan = copy_df.dropna(subset=['breakdown'])

# print(type(copy_df.iloc[246]['breakdown']))

index_with_ones = copy_df[copy_df['breakdown'] == '1'].index
# print(index_with_ones)

# Create a list of indices for the rows within the window
indices_in_window = []
for index in index_with_ones:
    window_indices = range(max(0, index - 2), min(len(copy_df), index + 3))  # Window of 5 rows, adjusted for boundaries
    indices_in_window.extend(window_indices)

# Remove duplicates and ensure the indices are within the valid range
indices_in_window = list(set(filter(lambda x: 0 <= x < len(copy_df), indices_in_window)))

# Select the rows within the window
breakdown_df= copy_df.iloc[indices_in_window]

# Now df_filtered contains a window of 5 rows around each row where 'column_name' has the value 1
breakdown_df.head(10)


,turn,start time,speaker,utterance,speech act,deepand,rrs,pop,ffb,conversation,directive_product,combined_utterance,breakdown
10,8.0,6:09,A1,"and then, we should be able to get it onto the...",Assertive,Predict,statement,Process,behavior,possibility,0,,NaN
11,8.0,6:09,A1,"But the open question is, we need to define so...",Directive,Formulate Alternative,request,Process,function,action,0,,NaN
12,8.0,6:09,A1,Is this the approach temperature? Is it the dp...,Directive,Formulate Alternative,request,Product,form,possibility,1,"and then, we should be able to get it onto the...",1
13,8.0,6:09,A1,where do we want to start? Or do we just want ...,Directive,Formulate Alternative,request,Process,form,possibility,0,,NaN
14,9.0,6:50,L0,yeah um so this this Software A CMMS integrati...,Directive,Describe,request,Product,function,clarification,1,Is this the approach temperature? Is it the dp...,1
15,10.0,7:01,A1,yeah. So…,Assertive,Describe,response,Product,function,clarification,0,,NaN
16,11.0,7:04,L0,So. yeah go ahead,Directive,?,statement,N/a,N/a,clarification,0,,NaN
98,71.0,16:34,A0,How about the installation and who is going to...,Directive,Describe,request,Process,function,possibility,0,,NaN
99,72.0,16:53,A1,Well interpret the data that's machine learning.,Assertive,Describe,response,Process,form,orientation,0,,NaN
100,72.0,16:53,A1,That’s why- where it's different than Company ...,Directive,Describe,response,Product,form,orientation,1,How about the installation and who is going to...,1


In [ ]:
copy_df.to_csv('/content/drive/MyDrive/Coterm/CS224V/generated_turn_has_breakdown_9.10', index=False)

legacy, use the one above when generating our own breakdown True/False

In [ ]:
# Find rows with breakdowns

# Assuming df is your DataFrame and 'column_name' is the column you're interested in
index_with_ones = copy_df[copy_df['speaker turn is part of breakdown?'] == 1.0].index

# Create a list of indices for the rows within the window
indices_in_window = []
for index in index_with_ones:
    window_indices = range(max(0, index - 2), min(len(copy_df), index + 3))  # Window of 5 rows, adjusted for boundaries
    indices_in_window.extend(window_indices)

# Remove duplicates and ensure the indices are within the valid range
indices_in_window = list(set(filter(lambda x: 0 <= x < len(copy_df), indices_in_window)))

# Select the rows within the window
breakdown_df= copy_df.iloc[indices_in_window]

# Now df_filtered contains a window of 5 rows around each row where 'column_name' has the value 1
breakdown_df.head(10)


,turn,start time,speaker,utterance,speech act,deepand,rrs,pop,ffb,conversation,directive_product,combined_utterance,breakdown
10,8.0,6:09,A1,"and then, we should be able to get it onto the...",Assertive,Predict,statement,Process,behavior,possibility,0,,NaN
11,8.0,6:09,A1,"But the open question is, we need to define so...",Directive,Formulate Alternative,request,Process,function,action,0,,NaN
12,8.0,6:09,A1,Is this the approach temperature? Is it the dp...,Directive,Formulate Alternative,request,Product,form,possibility,1,"and then, we should be able to get it onto the...",1
13,8.0,6:09,A1,where do we want to start? Or do we just want ...,Directive,Formulate Alternative,request,Process,form,possibility,0,,NaN
14,9.0,6:50,L0,yeah um so this this Software A CMMS integrati...,Directive,Describe,request,Product,function,clarification,1,Is this the approach temperature? Is it the dp...,1
15,10.0,7:01,A1,yeah. So…,Assertive,Describe,response,Product,function,clarification,0,,NaN
16,11.0,7:04,L0,So. yeah go ahead,Directive,?,statement,N/a,N/a,clarification,0,,NaN
98,71.0,16:34,A0,How about the installation and who is going to...,Directive,Describe,request,Process,function,possibility,0,,NaN
99,72.0,16:53,A1,Well interpret the data that's machine learning.,Assertive,Describe,response,Process,form,orientation,0,,NaN
100,72.0,16:53,A1,That’s why- where it's different than Company ...,Directive,Describe,response,Product,form,orientation,1,How about the installation and who is going to...,1


In [ ]:
# Initialize an empty DataFrame to store the grouped rows
result_df = pd.DataFrame(columns=['turn', 'speaker', 'utterance'])

current_group = []
for index, row in breakdown_df.iterrows():
    if not current_group or row['turn'] == current_group[-1]['turn'] or row['turn'] == current_group[-1]['turn'] + 1:
        current_group.append(row)
    else:
        # Concatenate rows in the current group and add to the result DataFrame
        result_row = {
            'turn': current_group[0]['turn'],
            'speaker': ', '.join(list(set([str(r['speaker']) for r in current_group if not str(r['speaker']) == 'nan']))),
            'utterance': ' '.join([r['utterance'] for r in current_group]),
        }
        result_df = result_df.append(result_row, ignore_index=True)
        current_group = [row]

# Concatenate the last group if it exists
if current_group:
    result_row = {
        'turn': current_group[0]['turn'],
        'speaker': ', '.join(list(set([r['speaker'] for r in current_group]))),
        'utterance': ' '.join([r['utterance'] for r in current_group]),
    }
    result_df = result_df.append(result_row, ignore_index=True)

print(len(result_df))
# Print the resulting DataFrame
result_df.head(20)

16


<ipython-input-10-2eaa911636ef>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_row, ignore_index=True)
<ipython-input-10-2eaa911636ef>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_row, ignore_index=True)
<ipython-input-10-2eaa911636ef>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_row, ignore_index=True)
<ipython-input-10-2eaa911636ef>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_row, ignore_index=True)
<ipython-input-10-2eaa911636ef>:15: FutureWarning: The frame.append method i

,turn,speaker,utterance
0,8.0,"A1, L0","and then, we should be able to get it onto the..."
1,71.0,"A0, A1",How about the installation and who is going to...
2,95.0,"A0, L0, A1",yeah behind the scene right? So And it should ...
3,132.0,"A0, L0, A1","yeah and we want to pursue the pilot. And A0, ..."
4,179.0,"A1, L0, A0",We have human learning And… yeah human learnin...
5,189.0,"A1, L0","you want to quickly show uh, show me what they..."
6,244.0,"A1, L0",so yeah i'm really curious about how Company Z...
7,290.0,"A0, UK, L0","[silence] I think the office, they do shut shu..."
8,313.0,"A0, L0, A1",Because ah let me show you the analysis. [clic...
9,325.0,"A1, A0","And you drove, you drive 12 hours. Then you ha..."


From here, we can pass in these conversation that involved breakdowns to the LLM, saying here is more information about them.

In [ ]:
breakdown_convos = ''
for ind, row in result_df.iterrows():
  breakdown_convos += row['speaker'] + ': ' + row['utterance'] + '\n'

print(breakdown_convos)

A1, L0: and then, we should be able to get it onto the production system, I think, within um one or two months. But the open question is, we need to define some meters somewhere so. kind of need to agree on where we're putting that. Is this the approach temperature? Is it the dp? is it, is it the Software C freezer monitoring, where do we want to start? Or do we just want to start with one thing to check it out? yeah um so this this Software A CMMS integration, we want to use the CMMS to generate the work order right? yeah. So… So. yeah go ahead
A0, A1: How about the installation and who is going to look at the data, interpret the data for us Well interpret the data that's machine learning. That’s why- where it's different than Company R right?wrong and kind of warn you  Company R they one is- even though it's wireless you still have to manually look at the data. Here they will, machine learning will put- it will see when something's  Yeah well without without looking at it does it com

In [ ]:
with open('/content/drive/MyDrive/Coterm/CS224V/breakdown_convos_9.10.txt', 'w') as txt_file:
  txt_file.write(breakdown_convos)

In [ ]:
!pip install --upgrade openai

In [ ]:
import openai
print(openai.VERSION)

1.3.6


##### make summary

In [ ]:
example_summary = "What the meeting was about:\n\
[high level summary]:\n\
1. topic 1...\n\
2. topic 2...\n\
3. ...\n\
What speakers committed to doing after the meeting:\n\
1. A1 is going to do ...\n\
2. ...\n\
Action items that need to be done:\n\
1. action item 1: action item description...\n\
2. action item 2: ...\n\
Conversation breakdowns:\n\
[summary of breakdowns]\n\
1. explanation of breakdown 1.\n\
2. explanation of breakdown 2...\n"

print(example_summary)

What the meeting was about:
[high level summary]:
1. topic 1...
2. topic 2...
3. ...
What speakers committed to doing after the meeting:
1. A1 is going to do ...
2. ...
Action items that need to be done:
1. action item 1: action item description...
2. action item 2: ...
Conversation breakdowns:
[summary of breakdowns]
1. explanation of breakdown 1.
2. explanation of breakdown 2...



In [ ]:
import pandas as pd

transcript_file_path = '/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/'
raw_df = pd.read_csv(transcript_file_path + '09.10mtg-Table 1.csv')


transcript_str = ''
for index, row in raw_df.iterrows():
  speaker = row['speaker']
  speaker_turn = row['speaker turn']
  transcript_str += str(speaker) + ': ' + str(speaker_turn) + '\n'


In [ ]:
import os
from openai import OpenAI
openai.organization = "add org"
openai.api_key = "add api key"

system_message = "You are a meeting summarizer. Summarize this meeting transcript and split it into sections: what the meeting was about, what speakers committed to doing after the meeting, \
what action items need to be done, and what conversation breakdowns occured. A breakdown is when there is some miscommunication that prevents the meeting from progressing. \
For each topic, provide details and context about that topic. For the action items, add explanations of the items such that someone who is somewhat unfamiliar with the domain could understand. \
Add lots of details, the summary does not have to be super concise. We also provide conversations in the meeting related to breakdowns for you to use to add more detail about the conversation breakdowns. \
When you find a conversation breakdown, explain why it is breakdown, then explain in detail the subject of that conversation."


# prompt += transcript_str[:len(transcript_str) // 3]

example_prompt = "Breakdown conversations:\n[breakdown conversations]\n\nTranscript:\n[sample transcript]"

prompt = "Breakdown conversations:\n" + breakdown_convos + '\n\nTranscript:\n' + transcript_str

example_summary = "What the meeting was about:\n\
[high level summary]:\n\
1. topic 1...\n\
2. topic 2...\n\
3. ...\n\
What speakers committed to doing after the meeting:\n\
1. A1 is going to do ...\n\
2. ...\n\
Action items that need to be done:\n\
1. action item 1: action item description...\n\
2. action item 2: ...\n\
Conversation breakdowns:\n\
[summary of breakdowns]\n\
1. explanation of breakdown 1.\n\
2. explanation of breakdown 2...\n"





# once you get the highlevel entites that were discussed, add detail about hwat they are about

# print(prompt)

# response = openai.Completion.create(
#   model='gpt-3.5-turbo-1106', # "text-davinci-003",  # Replace with gpt-4 model when available.
#   prompt=prompt,
#   temperature=0.5,
#   max_tokens=150,
#   n=1,
#   stop=None,
#   logprobs=None
# )

# print(response.choices[0].text.strip())

client = OpenAI(
    organization="add org",
    api_key="add api key"
)

response = client.chat.completions.create(
    model="gpt-4-1106-preview", # "gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": example_prompt},
        {"role": "assistant", "content": example_summary},
        {"role": "user", "content": prompt},
    ]
)

# Extract the assistant's reply from the response
assistant_reply = response.choices[0].message.content

# Print or use the assistant's reply
print(assistant_reply)

What the meeting was about:
The meeting primarily focused on several technical discussions:

1. Software A to CMMS Integration: The meeting addressed the progress of integrating Software A with the Computerized Maintenance Management System (CMMS) and deliberated over whether to start with work orders or service requests. The goal is to determine how condition-based maintenance will impact current maintenance processes.

2. Software C & Life Safety Sensors: Participants also discussed exploring Software C and handling potential fault alerts that could automatically generate work orders.

3. Company Z Pilot for Smart Monitoring: The team debated setting up a pilot project with Company Z to compare its machine learning capability with the existing wireless system from Company R. Metrics like bearing failure, lubrication issues, and other performance indicators would be compared to check the reliability and cost-effectiveness of Company Z’s technology.

4. Data Interpretation & Machine Le

In [ ]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.0 MB/s eta 0:00:00


In [ ]:
assistant_reply = assistant_reply.replace('*', '').replace('#', '')
print(assistant_reply)

What the meeting was about:
The meeting was focused on several interrelated topics involving software integration, sensor installation, machine learning analytics, and maintenance processes:

1. Integration of Software A with CMMS: The team discussed the progression of integrating Software A and CMMS (Computerized Maintenance Management System), which would allow CMMS to generate work orders based on data received, and there was a debate over whether to generate work orders or service requests.

2. Impact on Maintenance Protocols: The impacts of the shift from scheduled maintenance to condition-based maintenance (CBM) on existing maintenance protocols, costs, logistics, and vendor relationships.

3. Pilot Test for Software C: The possibility of setting up a hybrid scheduled and CBM system to test the integration, and how to proceed with a pilot test.

4. Discussion with Company Z: There was an evaluation of a potential pilot with Company Z for machine learning analysis of building equi

In [ ]:
from docx import Document

doc = Document()
doc.add_paragraph(assistant_reply)
doc.save('/content/drive/MyDrive/Coterm/CS224V/9.10_meeting_summary_GPT4.docx')

### Complex2

Goal is to making a meeting summary based solely on the breakdowns. Each bullet point is a conversation breakdown and the sub-bullet points add more detail

In [ ]:
# make raw input transcript
def get_raw_transcript(file_path):
  transcript_file_path = file_path  # '/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/09.10mtg-Table 1.csv'
  raw_df = pd.read_csv(transcript_file_path)

  raw_df['speaker'] = raw_df['speaker'].fillna('NA')
  raw_df = raw_df.astype(str)
  raw_df = raw_df.replace(['?', 'N/a', 'n/a'], 'NA')

  transcript_str = ''
  for index, row in raw_df.iterrows():
    speaker = row['speaker']
    speaker_turn = row['speaker turn']
    transcript_str += str(speaker) + ': ' + str(speaker_turn) + '\n'

  return transcript_str



In [ ]:
def make_labeled_transcript(input_path, output_path):

  structured_df = pd.read_csv(input_path)
  structured_df['breakdown'] = structured_df['breakdown'].fillna('0')
  structured_df['breakdown'] = structured_df['breakdown'].replace(1.0, '1')
  structured_df['breakdown'] = structured_df['breakdown'].replace(0.0, '0')
  structured_df['directive_product'] = structured_df['directive_product'].replace(0.0, '0')
  structured_df['directive_product'] = structured_df['directive_product'].replace(1.0, '1')
  structured_df['speaker'] = structured_df['speaker'].fillna('NA')
  structured_df = structured_df.astype(str)
  structured_df = structured_df.replace(['?', 'N/a', 'n/a'], 'NA')


  transcript = ''
  for ind, row in structured_df.iterrows():
    transcript += row['speaker'] + ' (' + row['speech act'] + ', ' + row['deepand'] + ', ' + row['pop'] + ', ' + row['breakdown'] + '): ' + row['utterance'] + '\n'

  # print(transcript)
  with open(output_path, 'w') as txt_file:
    txt_file.write(transcript)


In [ ]:
def get_labeled_meetings(file_path):
  with open(file_path, 'r') as txt_file:
    labeled_meeting_conversations = txt_file.read()
  return labeled_meeting_conversations

In [ ]:
system_message = "You are a meeting summarizer. \
Each line in the labeled meeting conversations is structured as follows:\n\
[Speaker Name] ([Speech Act Classification], [Deepand Classification], [POP Classification], [breakdown exists?]): [speaker utterance]\n\
Speech Acts can be classified as Directive, Commissive, Expressive, Declaration, or Assertive. Here are the definitions for each of them:\n\
Directive: Speaker tries to get someone to do something.\n\
Commissive: When someone commits to something.\n\
Expressive: Speaker expresses psychological state about specific affairs.\n\
Declaration: Speaker declares something.\n\
Assertive: Speaker utters soemething to describe an existing state of affairs.\n\n\
Deepand can be classified as Describe, Explain, Evaluate, Predict, Formulate Alternative, Negotiate, or Decide. Here are the definitions for each of them:\n\
Describe: Speaker relays what is explicit in a model, documentation, or other source of data or information.\n\
Explain: Speaker provides further insight or clarity into a matter or a position.\n\
Evaluate: Speaker assesses extent to which an option meets defined or inferred requirements, assesses options relative to one another, ranks alternatives.\n\
Predict: Speaker calculates or estimates a parameter value.\n\
Formulate Alternative: Speaker presents an alternative, talks out a possible option modifying the problem space by adding or combining alternate solutions, criteria, or requirements.\n\
Negotiate: Speaker sorts out who will do what or who is responsible for what; focuses on the actor(s) who will do something or who will be responsible for something; figuring out the logistics of someone or a group doing something.\n\
Decide: Speaker makes and/or confirms a choice, chooses an alternative or a solution.\n\n\
POP can be classified as Organization, Process, or Product. Here are the definitions for each of them:\n\
Organization: Speaker is talking about an organization.\n\
Process: Speaker is talking about a process.\n\
Product: speaker is talking about a product.\n\n\
Breakdown exists will be 1 when a breakdown exists in this utterance or around it and will be 0 if not.\n\n\
Read the transcript and look at speaker turns that voiced a breakdown. \
For each of these turns, draw context from surrounding utterances and knowledge from the rest of the transcript. \
Think about what the team gained from voicing the breakdown and discussing it. In the instance of misunderstandings, \
explain what understanding was gained about a topic, for example what data is in Software A, or how Software A works. \
Only include an action item sub-bullet point if one was found because of a breakdown or because it was discussed in close proximity to a breakdown.\
Each section should be a breakdown topic and the bullet points should be a detailed discussion about the breakdown in the way explained above. \
It is helpful to include speaker names, numbers, statistics, products, and/or entities that were mentioned. \
For example, a bullet point with a sub-bullet point could be:\n\
- L0 reviews his estimates for the cooling tower fan run hours and E0 highlights that he assumes \
1000 hours per year which implies only 3 hours run time per day. This seems low to E0.\n\
    - A0 shows L0 how to bring up the data in Software A.\n\
    - Action item: Z2 will schedule a meeting with M3 for them to discuss this issue."

print(system_message)

You are a meeting summarizer. Each line in the labeled meeting conversations is structured as follows:
[Speaker Name] ([Speech Act Classification], [Deepand Classification], [POP Classification], [breakdown exists?]): [speaker utterance]
Speech Acts can be classified as Directive, Commissive, Expressive, Declaration, or Assertive. Here are the definitions for each of them:
Directive: Speaker tries to get someone to do something.
Commissive: When someone commits to something.
Expressive: Speaker expresses psychological state about specific affairs.
Declaration: Speaker declares something.
Assertive: Speaker utters soemething to describe an existing state of affairs.

Deepand can be classified as Describe, Explain, Evaluate, Predict, Formulate Alternative, Negotiate, or Decide. Here are the definitions for each of them:
Describe: Speaker relays what is explicit in a model, documentation, or other source of data or information.
Explain: Speaker provides further insight or clarity into a m

In [ ]:
import os
from openai import OpenAI

def complex2_summarize(breakdown_convos, labeled_transcript):
  openai.organization = "add org"
  openai.api_key = "add api"

  # system_message = "You are a meeting summarizer. Summarize this meeting transcript and split it into sections: what the meeting was about, what speakers committed to doing after the meeting, \
  # what action items need to be done, and what conversation breakdowns occured. A breakdown is when there is some miscommunication that prevents the meeting from progressing. \
  # For each topic, provide details and context about that topic. For the action items, add explanations of the items such that someone who is somewhat unfamiliar with the domain could understand. \
  # Add lots of details, the summary does not have to be super concise. We also provide conversations in the meeting related to breakdowns for you to use to add more detail about the conversation breakdowns. \
  # When you find a conversation breakdown, explain why it is breakdown, then explain in detail the subject of that conversation."

  # system_message = "You are a meeting summarizer. Summarize the transcript by making one bullet point per conversation breakdown topic. \
  # For each breakdown, add sub bullet points about what the team learned from the breakdown, what actions did the team voice as necessary \
  # for moving forward, and/or what questions did the team still have. Here is also a list of conversations from the transcript that are likely \
  # to contain breakdowns. Think about referencing these to get the breakdowns, then reading other parts of the transcript to get more context \
  # and generate the sub bullet points."

#   system_message = "You are a meeting summarizer. Read the transcript and look at speaker turns that voiced a breakdown. \
# For each of these turns, draw context from surrounding utterances and knowledge from the rest of the transcript. \
# Think about what the team gained from voicing the breakdown and discussing it. In the instance of misunderstandings, \
# explain what understanding was gained about a topic, for example what data is in Software A, or how Software A works. \
# Only include an action item sub-bullet point if one was found because of a breakdown or because it was discussed in close proximity to a breakdown.\
# Each section should be a breakdown topic and the bullet points should be the information about the breakdown in the way explained above. \
# For example, a bullet point with a sub-bullet point could be:\n\
# - L0 reviews his estimates for the cooling tower fan run hours and E0 highlights that he assumes \
# 1000 hours per year which implies only 3 hours run time per day. This seems low to E0.\n\
#     - A0 shows L0 how to bring up the data in Software A.\n\
#     - Action item: Z2 will schedule a meeting with M3 for them to discuss this issue."

# Provided is a list of utterances that are likely to contain breakdowns, which you can use to think about what breakdowns occurred in the meeting."

  system_message = "You are a meeting summarizer. \
Each line in the labeled meeting conversations is structured as follows:\n\
[Speaker Name] ([Speech Act Classification], [Deepand Classification], [POP Classification], [breakdown exists?]): [speaker utterance]\n\
Speech Acts could be Directive, Commissive, Expressive, Declaration, or Assertive. Definitions:\n\
Directive: Speaker tries to get someone to do something.\n\
Commissive: When someone commits to something.\n\
Expressive: Speaker expresses psychological state about specific affairs.\n\
Declaration: Speaker declares something.\n\
Assertive: Speaker utters soemething to describe an existing state of affairs.\n\n\
Deepand could be Describe, Explain, Evaluate, Predict, Formulate Alternative, Negotiate, or Decide. Definitions:\n\
Describe: Speaker relays what is explicit in a model, documentation, or other source of data or information.\n\
Explain: Speaker provides further insight or clarity into a matter or a position.\n\
Evaluate: Speaker assesses extent to which an option meets defined or inferred requirements, assesses options relative to one another, ranks alternatives.\n\
Predict: Speaker calculates or estimates a parameter value.\n\
Formulate Alternative: Speaker presents an alternative, talks out a possible option modifying the problem space by adding or combining alternate solutions, criteria, or requirements.\n\
Negotiate: Speaker sorts out who will do what or who is responsible for what; focuses on the actor(s) who will do something or who will be responsible for something; figuring out the logistics of someone or a group doing something.\n\
Decide: Speaker makes and/or confirms a choice, chooses an alternative or a solution.\n\n\
POP could be Organization, Process, or Product. Definitions:\n\
Organization: Speaker is talking about an organization.\n\
Process: Speaker is talking about a process.\n\
Product: speaker is talking about a product.\n\n\
Breakdown exists will be 1 when a breakdown exists in this utterance or around it and will be 0 otherwise.\n\n\
Read the transcript and look at speaker turns that voiced a breakdown. \
For each turn with a breakdown, draw context from surrounding utterances and knowledge from the rest of the transcript. \
Think about what the team gained from voicing the breakdown and discussing it. In the instance of misunderstandings, \
explain what understanding was gained about a topic, for example what data is in Software A, or how Software A works. \
Only include an action item sub-bullet point if one was found because of a breakdown or because it was discussed in close proximity to a breakdown.\
Each section should be a breakdown topic and the bullet points should be a detailed discussion about the breakdown in the way explained above. \
Breakdown topics should only be about one discussion. Don't group discussion together unless they are very closely related. \
It is helpful to include speaker names, numbers, statistics, products, and/or entities that were mentioned. \
Provide a structured and detailed discussion based on these instructions. Provided are also conversations that are likely to have \
breakdowns, use them to think about the high-level breakdowns that occurred.\n\
An example bullet point with a sub-bullet point could be:\n\
- A0 reviews his estimates for the cooling tower fan run hours and L0 highlights that he assumes \
1000 hours per year which implies only 3 hours run time per day. This seems low to L0.\n\
    - A0 shows L0 how to bring up the data in Software A.\n\
    - Action item: A0 will schedule a meeting with L0 to continue working on this."

  # print(system_message)
  # example_prompt = "Breakdown conversations:\n[breakdown conversations]\n\nTranscript:\n[sample transcript]"

  # example_summary = "\
  # [breakdown topic 1]\n\
  # - [A1 ask for ...]\n\
  # - [B2 clarifies ...]\n\
  # "

  prompt = "Breakdown conversations:\n" + breakdown_convos + '\n\nTranscript:\n' + labeled_transcript # transcript_str (raw transcript)
  # prompt = "Transcript:\n" + labeled_meeting_conversations


  client = OpenAI(
      organization="add org",
      api_key="add api key"
  )

  response = client.chat.completions.create(
      model="gpt-4-1106-preview", # "gpt-3.5-turbo-1106",
      messages=[
          {"role": "system", "content": system_message},
          # {"role": "user", "content": example_prompt},
          # {"role": "assistant", "content": example_summary},
          {"role": "user", "content": prompt},
      ]
  )

  # Extract the assistant's reply from the response
  assistant_reply = response.choices[0].message.content

  # Print or use the assistant's reply
  print(assistant_reply)
  return assistant_reply

In [ ]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 2.9 MB/s eta 0:00:00


In [ ]:
def make_labeled_transcript_our_version(input_path, output_path):

  structured_df = pd.read_csv(input_path)
  structured_df['breakdown'] = structured_df['breakdown'].fillna('0')
  structured_df['breakdown'] = structured_df['breakdown'].replace(1.0, '1')
  structured_df['breakdown'] = structured_df['breakdown'].replace(0.0, '0')
  structured_df['directive_product'] = structured_df['directive_product'].replace(0.0, '0')
  structured_df['directive_product'] = structured_df['directive_product'].replace(1.0, '1')
  structured_df['speaker'] = structured_df['speaker'].fillna('NA')
  structured_df = structured_df.astype(str)
  structured_df = structured_df.replace(['?', 'N/a', 'n/a'], 'NA')


  transcript = ''
  for ind, row in structured_df.iterrows():
    transcript += row['speaker'] + ' (' + row['breakdown'] + '): ' + row['speaker turn'] + '\n'

  # print(transcript)
  with open(output_path, 'w') as txt_file:
    txt_file.write(transcript)


In [ ]:
import os
from openai import OpenAI

def complex2_summarize_our_version(breakdown_convos, labeled_transcript):
  openai.organization = "add org"
  openai.api_key = "add api key"

  # system_message = "You are a meeting summarizer. Summarize this meeting transcript and split it into sections: what the meeting was about, what speakers committed to doing after the meeting, \
  # what action items need to be done, and what conversation breakdowns occured. A breakdown is when there is some miscommunication that prevents the meeting from progressing. \
  # For each topic, provide details and context about that topic. For the action items, add explanations of the items such that someone who is somewhat unfamiliar with the domain could understand. \
  # Add lots of details, the summary does not have to be super concise. We also provide conversations in the meeting related to breakdowns for you to use to add more detail about the conversation breakdowns. \
  # When you find a conversation breakdown, explain why it is breakdown, then explain in detail the subject of that conversation."

  # system_message = "You are a meeting summarizer. Summarize the transcript by making one bullet point per conversation breakdown topic. \
  # For each breakdown, add sub bullet points about what the team learned from the breakdown, what actions did the team voice as necessary \
  # for moving forward, and/or what questions did the team still have. Here is also a list of conversations from the transcript that are likely \
  # to contain breakdowns. Think about referencing these to get the breakdowns, then reading other parts of the transcript to get more context \
  # and generate the sub bullet points."

#   system_message = "You are a meeting summarizer. Read the transcript and look at speaker turns that voiced a breakdown. \
# For each of these turns, draw context from surrounding utterances and knowledge from the rest of the transcript. \
# Think about what the team gained from voicing the breakdown and discussing it. In the instance of misunderstandings, \
# explain what understanding was gained about a topic, for example what data is in Software A, or how Software A works. \
# Only include an action item sub-bullet point if one was found because of a breakdown or because it was discussed in close proximity to a breakdown.\
# Each section should be a breakdown topic and the bullet points should be the information about the breakdown in the way explained above. \
# For example, a bullet point with a sub-bullet point could be:\n\
# - L0 reviews his estimates for the cooling tower fan run hours and E0 highlights that he assumes \
# 1000 hours per year which implies only 3 hours run time per day. This seems low to E0.\n\
#     - A0 shows L0 how to bring up the data in Software A.\n\
#     - Action item: Z2 will schedule a meeting with M3 for them to discuss this issue."

# Provided is a list of utterances that are likely to contain breakdowns, which you can use to think about what breakdowns occurred in the meeting."

  system_message = "You are a meeting summarizer. \
Each line in the labeled meeting conversations is structured as follows:\n\
[Speaker Name] ([breakdown exists?]): [speaker utterance]\n\
Breakdown exists will be 1 when a breakdown exists in this utterance or around it and will be 0 otherwise.\n\n\
Read the transcript and look at speaker turns that voiced a breakdown. \
For each turn with a breakdown, draw context from surrounding utterances and knowledge from the rest of the transcript. \
Think about what the team gained from voicing the breakdown and discussing it. In the instance of misunderstandings, \
explain what understanding was gained about a topic, for example what data is in Software A, or how Software A works. \
Only include an action item sub-bullet point if one was found because of a breakdown or because it was discussed in close proximity to a breakdown.\
Each section should be a breakdown topic and the bullet points should be a detailed discussion about the breakdown in the way explained above. \
Breakdown topics should only be about one discussion. Don't group discussion together unless they are very closely related. \
It is helpful to include speaker names, numbers, statistics, products, and/or entities that were mentioned. \
Provide a structured and detailed discussion based on these instructions. Provided are also conversations that are likely to have \
breakdowns, use them to think about the high-level breakdowns that occurred.\n\
An example bullet point with a sub-bullet point could be:\n\
- A0 reviews his estimates for the cooling tower fan run hours and L0 highlights that he assumes \
1000 hours per year which implies only 3 hours run time per day. This seems low to L0.\n\
    - A0 shows L0 how to bring up the data in Software A.\n\
    - Action item: A0 will schedule a meeting with L0 to continue working on this."

  # print(system_message)
  # example_prompt = "Breakdown conversations:\n[breakdown conversations]\n\nTranscript:\n[sample transcript]"

  # example_summary = "\
  # [breakdown topic 1]\n\
  # - [A1 ask for ...]\n\
  # - [B2 clarifies ...]\n\
  # "

  prompt = "Breakdown conversations:\n" + breakdown_convos + '\n\nTranscript:\n' + labeled_transcript # transcript_str (raw transcript)
  # prompt = "Transcript:\n" + labeled_meeting_conversations


  client = OpenAI(
      organization="add org",
      api_key="add api key"
  )

  response = client.chat.completions.create(
      model="gpt-4-1106-preview", # "gpt-3.5-turbo-1106",
      messages=[
          {"role": "system", "content": system_message},
          # {"role": "user", "content": example_prompt},
          # {"role": "assistant", "content": example_summary},
          {"role": "user", "content": prompt},
      ]
  )

  # Extract the assistant's reply from the response
  assistant_reply = response.choices[0].message.content

  # Print or use the assistant's reply
  print(assistant_reply)
  return assistant_reply

In [ ]:
from docx import Document

make_labeled_transcript_our_version('/content/drive/MyDrive/Coterm/CS224V/find_breakdowns_our_version_9.10.csv', '/content/drive/MyDrive/Coterm/CS224V/our_version_transcript_9.10.txt')

# transcript_str = get_raw_transcript('/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/09.10mtg-Table 1.csv')
transcript_str = get_labeled_meetings('/content/drive/MyDrive/Coterm/CS224V/our_version_transcript_9.10.txt')
# print(transcript_str)

with open('/content/drive/MyDrive/Coterm/CS224V/_9.10.txt', 'r') as file:
    breakdown_convos_paragraph = file.read()

# print(breakdown_convos_paragraph)
# breakdown_convos = ''

summary = complex2_summarize_our_version(breakdown_convos_paragraph, transcript_str)


doc = Document()
doc.add_paragraph(summary)
doc.save('/content/drive/MyDrive/Coterm/CS224V/our_version_9.10_meeting_summary.docx')

Defining Data for Software A and CMMS Integration

- The team discusses the testing of Software A integration with CMMS and the need to define specific meters to monitor. A1 mentions testing on the development system before production and asks L0 about what to monitor: approach temperature, dp (differential pressure), or the Software C freezer monitoring system.
    - The conversation clarifies that CMMS will be used to generate work orders based on data fed from Software A, introducing a significant change from scheduled to condition-based maintenance.
    - There is concern about the integration impacting existing processes, logistics, and budgets, with implications for Company P.
    - Action item: Discuss with V5 and V6 regarding Software C (suggested by A0) to have their insight on integration and impact on CMMS.

Potential Hybrid Approach for Condition-Based Maintenance

- A1 proposes a hybrid condition-based maintenance approach by monitoring dp (differential pressure) and gener

### Naive1

In [ ]:
import os
from openai import OpenAI

def naive1_summarize(transcript):
  openai.organization = "add org"
  openai.api_key = "add api key"

  # system_message = "You are a meeting summarizer. Summarize this meeting transcript and split it into sections: what the meeting was about, what speakers committed to doing after the meeting, \
  # what action items need to be done, and what conversation breakdowns occured. A breakdown is when there is some miscommunication that prevents the meeting from progressing. \
  # For each topic, provide details and context about that topic. For the action items, add explanations of the items such that someone who is somewhat unfamiliar with the domain could understand. \
  # Add lots of details, the summary does not have to be super concise. We also provide conversations in the meeting related to breakdowns for you to use to add more detail about the conversation breakdowns. \
  # When you find a conversation breakdown, explain why it is breakdown, then explain in detail the subject of that conversation."

  # system_message = "You are a meeting summarizer. Summarize the transcript by making one bullet point per conversation breakdown topic. \
  # For each breakdown, add sub bullet points about what the team learned from the breakdown, what actions did the team voice as necessary \
  # for moving forward, and/or what questions did the team still have. Here is also a list of conversations from the transcript that are likely \
  # to contain breakdowns. Think about referencing these to get the breakdowns, then reading other parts of the transcript to get more context \
  # and generate the sub bullet points."

#   system_message = "You are a meeting summarizer. Read the transcript and look at speaker turns that voiced a breakdown. \
# For each of these turns, draw context from surrounding utterances and knowledge from the rest of the transcript. \
# Think about what the team gained from voicing the breakdown and discussing it. In the instance of misunderstandings, \
# explain what understanding was gained about a topic, for example what data is in Software A, or how Software A works. \
# Only include an action item sub-bullet point if one was found because of a breakdown or because it was discussed in close proximity to a breakdown.\
# Each section should be a breakdown topic and the bullet points should be the information about the breakdown in the way explained above. \
# For example, a bullet point with a sub-bullet point could be:\n\
# - L0 reviews his estimates for the cooling tower fan run hours and E0 highlights that he assumes \
# 1000 hours per year which implies only 3 hours run time per day. This seems low to E0.\n\
#     - A0 shows L0 how to bring up the data in Software A.\n\
#     - Action item: Z2 will schedule a meeting with M3 for them to discuss this issue."

# Provided is a list of utterances that are likely to contain breakdowns, which you can use to think about what breakdowns occurred in the meeting."

#   system_message = "You are a meeting summarizer. Summarize this meeting transcript and split it into sections: \
# what the meeting was about, what speakers committed to doing after the meeting, and what action items need to be done. \
# For each topic, provide a detailed discussion about that topic with context. For the action items, add explanations of the items such \
# that someone who is somewhat unfamiliar with the domain could understand."


  system_message = "You are an AI meeting summarizer. You will analyze meeting content using a raw meeting transcript. \
Each section should be a topic of conversation, and the bullet points should be detailed discussions about the topic. \
Topics should only be about one discussion. Don't group discussions together unless they are very closely related. \
It is helpful to include speaker names, numbers, statistics, products, and/or entities that were mentioned.\
For example, a bullet point with a sub-bullet point could be:\n\
- Integration of Software B to pipeline\n\
    - A0 shows L0 how to see the data in Software B."



  # print(system_message)
  # example_prompt = "Breakdown conversations:\n[breakdown conversations]\n\nTranscript:\n[sample transcript]"

  # example_summary = "\
  # [breakdown topic 1]\n\
  # - [A1 ask for ...]\n\
  # - [B2 clarifies ...]\n\
  # "

  prompt = 'Transcript:\n' + transcript # transcript_str (raw transcript)
  # prompt = "Transcript:\n" + labeled_meeting_conversations


  client = OpenAI(
      organization="add org",
      api_key="add api key"
  )

  response = client.chat.completions.create(
      model="gpt-4-1106-preview", # "gpt-3.5-turbo-1106",
      messages=[
          {"role": "system", "content": system_message},
          # {"role": "user", "content": example_prompt},
          # {"role": "assistant", "content": example_summary},
          {"role": "user", "content": prompt},
      ]
  )

  # Extract the assistant's reply from the response
  assistant_reply = response.choices[0].message.content

  # Print or use the assistant's reply
  print(assistant_reply)
  return assistant_reply

In [ ]:
input_transcript = get_raw_transcript('/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/09.10mtg-Table 1.csv')

summary = naive1_summarize(input_transcript)
summary = summary.replace('*', '').replace('### ', '').replace('#', '')

doc = Document()
doc.add_paragraph(summary)
doc.save('/content/drive/MyDrive/Coterm/CS224V/naive1_9.10_meeting_summary.docx')

### Integration of Software A with CMMS (Computerized Maintenance Management System)
- A1 and L0 discussed the integration completion timeline and immediate next steps.
  - Integration approved for completion this week.
  - A1 to start testing on the development system next week.
  - Production system integration expected in one to two months.
- Choosing starting points for integration and measurements.
  - Need to define specific meters for tracking, with options including approach temperature (dp), and Software C freezer monitoring.
- Considering the initial scope of integration and possible pilot testing.
  - The group discussed whether to start with one particular measurement for initial testing.
  
### CMMS Work Orders vs. Service Requests
- Discussion on whether CMMS should automatically generate work orders or service requests based on data from Software A.
  - Service requests require verification by a person before becoming work orders.

### Impact on Existing Maintenance Proc

### Naive 1+

In [ ]:
import os
from openai import OpenAI

def naive1_plus_summarize(transcript):
  openai.organization = "add org"
  openai.api_key = "add api key"


  system_message = "You are an AI meeting summarizer. You will analyze meeting content using a raw meeting transcript and note what is \
learned from breakdowns or what actions are taken as a result of breakdowns. A breakdown is when there is some misommunication, lack \
of information, or needed clarification that prevents the meeting from progressing smoothly. \
Each section should be a topic of conversation, and the bullet points should be detailed discussions about the topic. \
Topics should only be about one discussion. Don't group discussions together unless they are very closely related. \
It is helpful to include speaker names, numbers, statistics, products, and/or entities that were mentioned.\
For example, a bullet point with a sub-bullet point could be:\n\
- Integration of Software B to pipeline\n\
    - A0 shows L0 how to see the data in Software B."

# system_message = "You are a meeting summarizer. Summarize this meeting transcript and split it into sections: what the meeting was about, what speakers committed to doing after the meeting, \
# what action items need to be done, and what conversation breakdowns occured. A breakdown is when there is some miscommunication that prevents the meeting from progressing. \
# For each topic, provide details and context about that topic. For the action items, add explanations of the items such that someone who is somewhat unfamiliar with the domain could understand. \
# Add lots of details, the summary does not have to be super concise."

  # print(system_message)
  # example_prompt = "Breakdown conversations:\n[breakdown conversations]\n\nTranscript:\n[sample transcript]"

  # example_summary = "\
  # [breakdown topic 1]\n\
  # - [A1 ask for ...]\n\
  # - [B2 clarifies ...]\n\
  # "

  prompt = 'Transcript:\n' + transcript # transcript_str (raw transcript)
  # prompt = "Transcript:\n" + labeled_meeting_conversations


  client = OpenAI(
      organization="add org",
      api_key="add api key"
  )

  response = client.chat.completions.create(
      model="gpt-4-1106-preview", # "gpt-3.5-turbo-1106",
      messages=[
          {"role": "system", "content": system_message},
          # {"role": "user", "content": example_prompt},
          # {"role": "assistant", "content": example_summary},
          {"role": "user", "content": prompt},
      ]
  )

  # Extract the assistant's reply from the response
  assistant_reply = response.choices[0].message.content

  # Print or use the assistant's reply
  print(assistant_reply)
  return assistant_reply

In [ ]:
input_transcript = get_raw_transcript('/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/07.16mtg-Table 1.csv')

summary = naive1_plus_summarize(input_transcript)
summary = summary.replace('*', '').replace('### ', '').replace('#', '')

doc = Document()
doc.add_paragraph(summary)
doc.save('/content/drive/MyDrive/Coterm/CS224V/naive1_plus_7.16_meeting_summary.docx')

**Discussion on Data Analysis and Trend Tracking for Building Alpha**

- Tracking Pressure Trends with Software A
  - L0 initiated the discussion about using Software A to track pressure trends for filter changes.
  - A1 experienced a technical issue viewing the shared screen, which is a breakdown. This was resolved by other members confirming they could see the screen and guiding A1 to access the data independently.
  - L0 proposed averaging data points weekly to better analyze trends. There was a brief communication breakdown when A1 temporarily lost connection.
  - The team agreed to implement a running average of data and to capture four data points per month for various air handling units. M0's repair work on unit 27 and the data trends were reviewed.
  - A0 clarified that spikes in the data would be filtered out in the average and suggested specifying data points only during operational hours to avoid misleading trends.

**Condition Based Maintenance (CBM) Strategy and Software U

### Attempt to find breakdowns from raw transcript

In [ ]:
import os
from openai import OpenAI

def find_directive_product(line):
  openai.organization = "add org"
  openai.api_key = "add api key"


  system_message = "You are detecting if an input utterance from a meeting is directive and whether or not \
the utterance discusses a product. An utterance is directive if the speaker tries to get someone to do something. \
An utterance is discussing a product when the speaker is talking about a product. If the utterance is directive and \
discusses a product, output 1, otherwise 0. Do not output anything else."

  prompt = 'Utterance:\n' + line


  client = OpenAI(
      organization="add org",
      api_key="add api key"
  )

  response = client.chat.completions.create(
      model="gpt-4-1106-preview", # "gpt-3.5-turbo-1106",
      messages=[
          {"role": "system", "content": system_message},
          # {"role": "user", "content": example_prompt},
          # {"role": "assistant", "content": example_summary},
          {"role": "user", "content": prompt},
      ]
  )

  # Extract the assistant's reply from the response
  assistant_reply = response.choices[0].message.content

  # Print or use the assistant's reply
  # print(assistant_reply)
  return assistant_reply

In [ ]:
import pandas as pd

# input_transcript = get_raw_transcript('/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/09.10mtg-Table 1.csv')
input_df = pd.read_csv('/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/09.10mtg-Table 1.csv')

input_df['speaker'] = input_df['speaker'].fillna('NA')
input_df = input_df.astype(str)
input_df = input_df.replace(['?', 'N/a', 'n/a'], 'NA')

input_df['directive_product'] = ''

for index, row in input_df.iterrows():
  speaker = row['speaker']
  speaker_turn = row['speaker turn']
  input_str = str(speaker) + ': ' + str(speaker_turn)
  output = find_directive_product(input_str)
  input_df.at[index, 'directive_product'] = output

input_df.head(30)


KeyboardInterrupt: ignored

In [ ]:

input_df['directive_product'] = input_df['directive_product'].replace(0.0, '0')
input_df['directive_product'] = input_df['directive_product'].replace(1.0, '1')
input_df['speaker'] = input_df['speaker'].fillna('NA')
input_df['speaker'] = input_df['speaker'].astype(str)
input_df = input_df.astype(str)
input_df = input_df.replace(['?', 'N/a', 'n/a'], 'NA')

input_df.to_csv('/content/drive/MyDrive/Coterm/CS224V/identify_code_9.10.csv')

In [ ]:
# Assuming df is your DataFrame

# Function to concatenate utterances
def concatenate_utterances_our_version(index, df):
  # Handle the edge cases where the index is at the start or end of the DataFrame
  start_index = max(index - 8, 0)
  end_index = min(index + 8, len(df))
  # Concatenate the utterances
  return ' '.join(df.loc[start_index:end_index, 'speaker turn'])

# Function to concatenate utterances
def concatenate_breakdown_context_our_version(index, df):
  # Handle the edge cases where the index is at the start or end of the DataFrame
  start_index = max(index - 2, 0)
  end_index = min(index + 2, len(df))
  # Concatenate the utterances
  return ' '.join(df.loc[start_index:end_index, 'speaker turn'])

# Function to concatenate utterances
def concatenate_context_speakers_our_version(index, df):
  # Handle the edge cases where the index is at the start or end of the DataFrame
  start_index = max(index - 2, 0)
  end_index = min(index + 2, len(df))
  # Concatenate the utterances
  return ', '.join(df.loc[start_index:end_index, 'speaker'])


def make_directive_product_windows_our_version(structured_df):
  # Create a new column and initialize with empty strings
  structured_df['combined_utterance'] = ''
  structured_df['breakdown_context'] = ''
  structured_df['context_speakers'] = ''

  # Iterate over rows
  for index, row in structured_df.iterrows():
    # Check if 'Directive + Product' is TRUE
    if row['directive_product'] == 1:
      structured_df.at[index, 'combined_utterance'] = concatenate_utterances_our_version(index, structured_df)
      structured_df.at[index, 'breakdown_context'] = concatenate_breakdown_context_our_version(index, structured_df)
      structured_df.at[index, 'context_speakers'] = concatenate_context_speakers_our_version(index, structured_df)

  return structured_df

  # The df now contains a new column 'combined_utterance' with the concatenated utterances
  # structured_df.head(20)

In [ ]:
!pip install --upgrade openai

In [ ]:
# for each row that has directive and product, ask LLM if it contains a breakdown.
import os
from openai import OpenAI
import openai
import numpy as np


def determine_breakdown_our_version(structured_df):
  openai.organization = "add org"
  openai.api_key = "add api key"


  client = OpenAI(
      organization="add org",
      api_key="add api key"
  )

  system_message = "You are finding breakdowns in meeting conversations. \
A breakdown in a conversation exists when there is a miscommunication, a need for clarification, or a gap in information that prevents the meeting from progressing smoothly. \
Determine if an utterance contains a breakdown by reading the context conversation, then decide whether the utterance is a breakdown. \
Output a 1 if there is a breakdown, otherwise 0."

  structured_df['breakdown'] = np.nan

  copy_df = structured_df

  for index, row in copy_df.iterrows():
    if row['directive_product'] == 1:
      prompt = "Utterance with potential breakdown: " + row['speaker turn'] + '\n\n'
      prompt += "Context conversation: " + row['combined_utterance']

      # print(prompt)


      response = client.chat.completions.create(
          model="gpt-4-1106-preview", # "gpt-3.5-turbo-1106",
          messages=[
              {"role": "system", "content": system_message},
              {"role": "user", "content": prompt},
          ]
      )

      # Extract the assistant's reply from the response
      assistant_reply = response.choices[0].message.content
      # print(assistant_reply)
      # print('-----------------------------------------')
      # row['breakdown'] = assistant_reply
      copy_df.at[index, 'breakdown'] = assistant_reply


  # copy_df.head(130)
  copy_df['breakdown'] = copy_df['breakdown'].astype(str)
  return copy_df


In [ ]:
import pandas as pd
input_df = pd.read_csv('/content/drive/MyDrive/Coterm/CS224V/identify_code_9.10.csv')
input_df = make_directive_product_windows_our_version(input_df)
input_breakdown_df = determine_breakdown_our_version(input_df)
input_breakdown_df.head(20)
input_breakdown_df.to_csv('/content/drive/MyDrive/Coterm/CS224V/find_breakdowns_our_version_9.10.csv')

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

def evaluate_breakdown(gold_file_path, generated_file_path):
  # '/content/drive/MyDrive/Coterm/CS224V/Bi-Weekly-Labeled/9.10mtg.csv'
  gold_structured_df = pd.read_csv(gold_file_path)
  generated_structured_df = pd.read_csv(generated_file_path)

  # for the gold, mark all turns that have one utterance with breakdown true as all breakdown
  gold_structured_df['speaker turn is part of breakdown?'] = gold_structured_df['speaker turn is part of breakdown?'].apply(lambda x: 1 if x == 1 else 0)
  gold_structured_df['speaker turn is part of breakdown?'] = gold_structured_df.groupby('turn')['speaker turn is part of breakdown?'].transform('max')
  gold_structured_df['speaker turn is part of breakdown?'] = gold_structured_df['speaker turn is part of breakdown?'].fillna(0)
  gold_structured_df = gold_structured_df.groupby('turn')['speaker turn is part of breakdown?'].first().reset_index()
  # return gold_structured_df
  # print(gold_structured_df.iloc[-1])
  gold_structured_df = gold_structured_df.drop(gold_structured_df.index[-1])

  # for the generated, mark all turns taht have one utterance with breakdown true as all breakdown
  generated_structured_df['breakdown'] = generated_structured_df['breakdown'].apply(lambda x: 1 if x == 1 else 0)
  generated_structured_df['breakdown'] = generated_structured_df.groupby('turn')['breakdown'].transform('max')
  generated_structured_df['breakdown'] = generated_structured_df['breakdown'].fillna(0)
  # print(generated_structured_df.iloc[-1])




  # return generated_structured_df

  accuracy = accuracy_score(gold_structured_df['speaker turn is part of breakdown?'], generated_structured_df['breakdown'])
  precision = precision_score(gold_structured_df['speaker turn is part of breakdown?'], generated_structured_df['breakdown'])
  recall = recall_score(gold_structured_df['speaker turn is part of breakdown?'], generated_structured_df['breakdown'])
  f1 = f1_score(gold_structured_df['speaker turn is part of breakdown?'], generated_structured_df['breakdown'])

  print('accuracy:', accuracy)
  print('precision:', precision)
  print('recall:', recall)
  print('f1:', f1)

generated_structured_df = evaluate_breakdown('/content/drive/MyDrive/Coterm/CS224V/Bi-Weekly-Labeled/9.10mtg.csv', '/content/drive/MyDrive/Coterm/CS224V/find_breakdowns_our_version_9.10.csv')
# gold_structured_df.iloc[160:180]
# generated_structured_df.tail(20)

accuracy: 0.9308035714285714
precision: 0.2857142857142857
recall: 0.07142857142857142
f1: 0.1142857142857143


### Naive2

In [ ]:
system_message = "You are a meeting summarizer. Read the transcript and look at speaker turns that voiced a breakdown. \
A breakdown is when there is some miscommunication , lack of information, or needed clarification that prevents the meeting from progressing smoothly. \
When you read a turn that voices a breakdown, draw context from surrounding turns and knowledge from the rest of the transcript. \
Think about what the team gained from voicing the breakdown and discussing it. In the instance of misunderstandings, \
explain what understanding was gained about a topic, for example what data is in Software A, or how Software A works. \
Only include an action item sub-bullet point if one was found because of a breakdown or because it was discussed in close proximity to a breakdown. \
Each section should be a breakdown topic and the bullet points should be a detailed discussion about the breakdown in the way explained above. \
Breakdown topics should only be about one discussion. Don't group discussions together unless they are very closely related. \
It is helpful to include speaker names, numbers, statistics, products, and/or entities that were mentioned. \
Provide a structured and detailed discussion based on these instructions. \
An example bullet point with a sub-bullet point could be:\n\
- A0 reviews his estimates for the cooling tower fan run hours and L0 highlights that he assumes \
1000 hours per year which implies only 3 hours run time per day. This seems low to L0.\n\
    - A0 shows L0 how to bring up the data in Software A.\n\
    - Action item: A0 will schedule a meeting with L0 to continue working on this."

print(system_message)

You are a meeting summarizer. Read the transcript and look at speaker turns that voiced a breakdown. A breakdown is when there is some miscommunication , lack of information, or needed clarification that prevents the meeting from progressing smoothly. When you read a turn that voices a breakdown, draw context from surrounding turns and knowledge from the rest of the transcript. Think about what the team gained from voicing the breakdown and discussing it. In the instance of misunderstandings, explain what understanding was gained about a topic, for example what data is in Software A, or how Software A works. Only include an action item sub-bullet point if one was found because of a breakdown or because it was discussed in close proximity to a breakdown.Each section should be a breakdown topic and the bullet points should be a detailed discussion about the breakdown in the way explained above. Breakdown topics should only be about one discussion. Don't group discussions together unless 

In [ ]:
import os
import openai
from openai import OpenAI

def naive2_summarize(transcript):
  openai.organization = "add org"
  openai.api_key = "add api key"


#   system_message = "You are an AI meeting summarizer. You will analyze meeting content using a raw meeting transcript and note what is \
# learned from breakdowns or what actions are taken as a result of breakdowns. A breakdown is when there is some miscommunication, lack \
# of information, or needed clarification that prevents the meeting from progressing smoothly. \
# Each section should be a topic of conversation, and the bullet points should be detailed discussions about the topic. \
# Topics should only be about one discussion. Don't group discussions together unless they are very closely related. \
# It is helpful to include speaker names, numbers, statistics, products, and/or entities that were mentioned.\
# For example, a bullet point with a sub-bullet point could be:\n\
# - Integration of Software B to pipeline\n\
#     - A0 shows L0 how to see the data in Software B."


  system_message = "You are a meeting summarizer. Read the transcript and look at speaker turns that voiced a breakdown. \
A breakdown is when there is some miscommunication , lack of information, or needed clarification that prevents the meeting from progressing smoothly. \
When you read a turn that voices a breakdown, draw context from surrounding turns and knowledge from the rest of the transcript. \
Think about what the team gained from voicing the breakdown and discussing it. In the instance of misunderstandings, \
explain what understanding was gained about a topic, for example what data is in Software A, or how Software A works. \
Only include an action item sub-bullet point if one was found because of a breakdown or because it was discussed in close proximity to a breakdown. \
Each section should be a breakdown topic and the bullet points should be a detailed discussion about the breakdown in the way explained above. \
Breakdown topics should only be about one discussion. Don't group discussions together unless they are very closely related. \
It is helpful to include speaker names, numbers, statistics, products, and/or entities that were mentioned. \
Provide a structured and detailed discussion based on these instructions. \
An example bullet point with a sub-bullet point could be:\n\
- A0 reviews his estimates for the cooling tower fan run hours and L0 highlights that he assumes \
1000 hours per year which implies only 3 hours run time per day. This seems low to L0.\n\
    - A0 shows L0 how to bring up the data in Software A.\n\
    - Action item: A0 will schedule a meeting with L0 to continue working on this."


  prompt = 'Transcript:\n' + transcript


  client = OpenAI(
      organization="add org",
      api_key="add api key"
  )

  response = client.chat.completions.create(
      model="gpt-4-1106-preview", # "gpt-3.5-turbo-1106",
      messages=[
          {"role": "system", "content": system_message},
          # {"role": "user", "content": example_prompt},
          # {"role": "assistant", "content": example_summary},
          {"role": "user", "content": prompt},
      ]
  )

  # Extract the assistant's reply from the response
  assistant_reply = response.choices[0].message.content

  # Print or use the assistant's reply
  print(assistant_reply)
  return assistant_reply

In [ ]:
import pandas as pd
from docx import Document

input_transcript = get_raw_transcript('/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/09.10mtg-Table 1.csv')

summary = naive2_summarize(input_transcript)
summary = summary.replace('*', '').replace('### ', '').replace('#', '')

doc = Document()
doc.add_paragraph(summary)
doc.save('/content/drive/MyDrive/Coterm/CS224V/naive2_9.10_meeting_summary.docx')

Software A to CMMS Integration and Defining Parameters for Data Transfer
- There is confusion concerning where to start with the integration of Software A and the computerized maintenance management system (CMMS) and what parameters to measure and monitor.
    - A1 details that while Software A will send data to CMMS, it will not generate work orders; instead, the CMMS will decide when to generate work orders or service requests based on parameters like the differential pressure (dp) data.
    - L0 voices concerns about the decision to start generating work orders or service requests, how it impacts the maintenance schedule of Company P, which is currently scheduled-based maintenance, and the implications on budget, logistics, vendor relationships, and supply chain.
    - A1 and L0 discuss the potential for a hybrid system that monitors dp but maintains a schedule, perhaps with increased intervals between filter changes. This reflects a need for a pilot test to ease Company P into the 

### Get token lengths of each transcript

In [ ]:
!pip install --upgrade tiktoken

In [ ]:
import pandas as pd
# from tiktoken import count_tokens
# import tiktoken

dates = ['07.16', '07.30', '08.13', '08.27', '09.10']

for date in dates:
  transcript_df = pd.read_csv('/content/drive/MyDrive/Coterm/CS224V/cbm_biweekly_transcripts/' +  date + 'mtg-Table 1.csv')
  full_transcript_str = transcript_df['speaker turn'].str.cat(sep=' ')
  # token_length = tiktoken.count_tokens(full_transcript_str)
  # print(f'Date: {date}, Length: {token_length}')
  with open('/content/drive/MyDrive/Coterm/CS224V/transcript_' + date + '.txt', 'w') as file:
    file.write(full_transcript_str)

